In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [5]:


data = pd.read_csv("../data/vpufs_reduced_features.csv")

data = data.dropna(axis=1)
data = data.loc[:, ~data.columns.duplicated()]
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()

# 3. Extract features and labels
labels = data.iloc[:, -1].values
features_df = data.drop(data.columns[-1], axis=1)


print(data.shape)

print(data.head())


(624, 1000)
    586        384    87    751    81       262   697   581   364   383  ...  \
0  52.5  29.600000  65.4  11.20  22.5  3.580000  15.3  21.8  2.95  25.1  ...   
1  63.6  42.800000  68.7  12.00  27.3  4.620000  20.2  27.7  3.27  31.9  ...   
2  47.6   1.505021  75.8  10.20  32.6  3.310000  16.2  20.5  5.22  28.8  ...   
3  59.2  40.100000  62.1  14.00  27.2  3.300000  18.9  30.9  5.48  26.5  ...   
4  40.9  37.500000  74.7   4.61  32.9  1.510252  16.9  24.3  1.56  30.0  ...   

     884   634    29   172   263    45   741   378        906    840  
0  3.050  14.3  5.92  6.76  11.3  13.6  1.99  64.1  32.300000  12.70  
1  1.660  18.3  6.83  8.31  15.7  20.9  2.31  47.6  30.400000   8.12  
2  0.625  15.6  9.65  7.63  15.8  19.2  1.86  43.5  23.800000   8.85  
3  3.580  16.1  7.15  6.30  11.3  19.6  2.15  54.8  31.100000  10.80  
4  1.170  15.9  9.72  5.24  14.4  18.5  2.15  42.1   1.510252   9.81  

[5 rows x 1000 columns]


In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

def normalized_euclidean_similarity(X):
    """ Compute (1 - normalized Euclidean distance) similarity matrix """
    # Normalize data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    # Euclidean distances
    distances = cdist(X_scaled, X_scaled, metric='euclidean')
    # Normalize distances to [0, 1]
    max_dist = distances.max()
    normalized_dist = distances / max_dist
    # Convert to similarity
    similarity = 1 - normalized_dist
    return similarity

def build_clusters_with_overlap(similarity_matrix, r=0.9):
    """ 
    Identify neighbors based on similarity threshold
    ALLOWS samples to appear in multiple clusters for better merging
    """
    n_samples = similarity_matrix.shape[0]
    clusters = []
    densities = []
    
    # For each sample, create a cluster with all its similar neighbors
    for i in range(n_samples):
        # Find neighbors based on similarity threshold
        neighbors = np.where(similarity_matrix[i] >= r)[0]
        
        if len(neighbors) > 0:
            clusters.append(neighbors.tolist())
            densities.append(len(neighbors))
    
    return clusters, densities, similarity_matrix

# def merge_clusters_by_similarity(clusters, similarity_matrix, threshold=0.7):
#     """
#     Merge clusters based on center-to-center similarity
#     """
#     print(f"Starting with {len(clusters)} clusters")
    
#     # Calculate cluster centers (most central sample in each cluster)
#     centers = []
#     for cluster in clusters:
#         if not cluster:
#             centers.append(-1)  # Mark empty clusters
#             continue
            
#         # For each cluster, find the most central sample
#         best_center = -1
#         best_avg_sim = -1
        
#         for sample in cluster:
#             # Average similarity to other samples in cluster
#             if len(cluster) > 1:
#                 avg_sim = np.mean([similarity_matrix[sample, other] for other in cluster])
#             else:
#                 avg_sim = 1.0  # Single-sample cluster
                
#             if avg_sim > best_avg_sim:
#                 best_avg_sim = avg_sim
#                 best_center = sample
                
#         centers.append(best_center)
    
#     # Convert clusters to sets for easier operations
#     cluster_sets = [set(cluster) for cluster in clusters]
    
#     # Keep track of which clusters have been merged into others
#     merged_into = [-1] * len(cluster_sets)  # -1 means not merged
    
#     # Check every pair of clusters
#     # print("\n--- Merging by Center Similarity ---")
#     merges_performed = 0
    
#     while True:
#         best_sim = threshold
#         best_i, best_j = -1, -1
        
#         # Find the most similar pair of clusters
#         for i in range(len(cluster_sets)):
#             if merged_into[i] != -1 or centers[i] == -1:  # Skip if merged or empty
#                 continue
                
#             for j in range(i+1, len(cluster_sets)):
#                 if merged_into[j] != -1 or centers[j] == -1:  # Skip if merged or empty
#                     continue
                
#                 # Calculate similarity between centers
#                 center_sim = similarity_matrix[centers[i], centers[j]]
                
#                 if center_sim > best_sim:
#                     best_sim = center_sim
#                     best_i, best_j = i, j
        
#         # If no pair found above threshold, we're done
#         if best_i == -1:
#             break

        
#         cluster_sets[best_i].update(cluster_sets[best_j])  # Add all elements from j to i
#         merged_into[best_j] = best_i  # Mark j as merged into i
#         merges_performed += 1

        
#     print(f"Performed {merges_performed} similarity-based merges")
    
#     # Collect final clusters (those not merged into others)
#     final_clusters = []
#     for i, merged_status in enumerate(merged_into):
#         if merged_status == -1 and centers[i] != -1:  # Not merged into another and not empty
#             final_clusters.append(list(cluster_sets[i]))
    
    
#     # Sort by size
#     final_clusters.sort(key=len, reverse=True)
    
    
#     return final_clusters


def merge_clusters_by_similarity(clusters, similarity_matrix, threshold=0.7, handle_small_clusters=True):
    """
    Merge clusters based on center-to-center similarity and handle small clusters
    """
    print(f"Starting with {len(clusters)} clusters")
    
    # Calculate cluster centers (most central sample in each cluster)
    centers = []
    for cluster in clusters:
        if not cluster:
            centers.append(-1)  # Mark empty clusters
            continue
            
        # For each cluster, find the most central sample
        best_center = -1
        best_avg_sim = -1
        
        for sample in cluster:
            # Average similarity to other samples in cluster
            if len(cluster) > 1:
                avg_sim = np.mean([similarity_matrix[sample, other] for other in cluster])
            else:
                avg_sim = 1.0  # Single-sample cluster
                
            if avg_sim > best_avg_sim:
                best_avg_sim = avg_sim
                best_center = sample
                
        centers.append(best_center)
    
    # Convert clusters to sets for easier operations
    cluster_sets = [set(cluster) for cluster in clusters]
    
    # Keep track of which clusters have been merged into others
    merged_into = [-1] * len(cluster_sets)  # -1 means not merged
    
    # First phase: merge by center similarity (with recursive propagation)
    merges_performed = 0
    
    # This set will track which clusters need to be checked again after a merge
    clusters_to_check = set(range(len(cluster_sets)))
    
    while clusters_to_check:
        # Get a cluster to check
        i = clusters_to_check.pop()
        
        # Skip if this cluster was merged into another
        if merged_into[i] != -1 or centers[i] == -1:
            continue
            
        # Find most similar cluster to merge with
        best_sim = threshold
        best_j = -1
        
        for j in range(len(cluster_sets)):
            if i == j or merged_into[j] != -1 or centers[j] == -1:
                continue
                
            # Calculate similarity between centers
            center_sim = similarity_matrix[centers[i], centers[j]]
            
            if center_sim > best_sim:
                best_sim = center_sim
                best_j = j
        
        # If found a cluster to merge with
        if best_j != -1:
            # Merge clusters
            cluster_sets[i].update(cluster_sets[best_j])
            merged_into[best_j] = i
            
            # Add this cluster back to check set since it changed
            clusters_to_check.add(i)
            
            # Update center for the merged cluster
            if len(cluster_sets[i]) > 0:
                best_center = -1
                best_avg_sim = -1
                
                for sample in cluster_sets[i]:
                    avg_sim = np.mean([similarity_matrix[sample, other] for other in cluster_sets[i]])
                    if avg_sim > best_avg_sim:
                        best_avg_sim = avg_sim
                        best_center = sample
                
                centers[i] = best_center
                
            merges_performed += 1
    
    print(f"Performed {merges_performed} recursive similarity-based merges")
    
    # Second phase: handle small clusters
    if handle_small_clusters:
        small_cluster_threshold = 3  # Consider clusters with <= 3 samples as small
        
        # Identify small clusters
        small_cluster_indices = []
        for i in range(len(cluster_sets)):
            if merged_into[i] == -1 and centers[i] != -1 and len(cluster_sets[i]) <= small_cluster_threshold:
                small_cluster_indices.append(i)
        
        print(f"Found {len(small_cluster_indices)} small clusters to potentially merge")
        
        # Try to merge each small cluster into its most similar larger cluster
        additional_merges = 0
        for i in small_cluster_indices:
            if merged_into[i] != -1:  # Skip if already merged
                continue
                
            best_sim = 0.3  # Lower threshold for small clusters
            best_j = -1
            
            # Find most similar larger cluster
            for j in range(len(cluster_sets)):
                if merged_into[j] != -1 or centers[j] == -1 or i == j:
                    continue
                    
                if len(cluster_sets[j]) <= small_cluster_threshold:
                    continue  # Skip other small clusters
                
                # Calculate average similarity between this small cluster and the larger one
                avg_sim = 0
                count = 0
                for sample_i in cluster_sets[i]:
                    for sample_j in cluster_sets[j]:
                        avg_sim += similarity_matrix[sample_i, sample_j]
                        count += 1
                
                if count > 0:
                    avg_sim /= count
                    
                    if avg_sim > best_sim:
                        best_sim = avg_sim
                        best_j = j
            
            # Merge if found a good match
            if best_j != -1:
                cluster_sets[best_j].update(cluster_sets[i])
                merged_into[i] = best_j
                additional_merges += 1
        
        print(f"Performed {additional_merges} additional merges for small clusters")
    
    # Collect final clusters (those not merged into others)
    final_clusters = []
    for i, merged_status in enumerate(merged_into):
        if merged_status == -1 and centers[i] != -1:  # Not merged into another and not empty
            final_clusters.append(list(cluster_sets[i]))
    
    # Sort by size
    final_clusters.sort(key=len, reverse=True)
    
    print(f"Final number of clusters: {len(final_clusters)}")
    
    return final_clusters

def remove_duplicates(clusters, similarity_matrix):
    """
    Ensure each sample appears in only one cluster - the one where it has highest similarity
    """
    n_samples = similarity_matrix.shape[0]
    
    # Track which cluster each sample is assigned to
    sample_to_cluster = [-1] * n_samples
    sample_similarities = [0.0] * n_samples
    
    # Process each cluster
    for cluster_idx, cluster in enumerate(clusters):
        for sample in cluster:
            # Calculate average similarity to this cluster
            cluster_sim = np.mean([similarity_matrix[sample, other] for other in cluster])
            
            # If not yet assigned or better similarity, assign to this cluster
            if sample_to_cluster[sample] == -1 or cluster_sim > sample_similarities[sample]:
                sample_to_cluster[sample] = cluster_idx
                sample_similarities[sample] = cluster_sim
    
    # Create deduplicated clusters
    deduplicated = [[] for _ in range(len(clusters))]

  

    # Assign each sample to its best cluster
    for sample, cluster_idx in enumerate(sample_to_cluster):
        if cluster_idx != -1:  # Skip unassigned samples
            deduplicated[cluster_idx].append(sample)
    
    # Remove empty clusters
    deduplicated = [cluster for cluster in deduplicated if cluster]
    
    # Sort by size
    deduplicated.sort(key=len, reverse=True)
    
    return deduplicated



In [7]:


def run_pipeline(data: pd.DataFrame, labels=None, k=1000, r=0.9, merge_threshold=0.7):
    print("Step 2: Computing similarity matrix...")
    similarity = normalized_euclidean_similarity(data)
    
    print("Step 3: Identifying neighbors and densities...")
    clusters, densities, similarity = build_clusters_with_overlap(similarity, r=r)

    # # 🚩 Print initial clusters before merging
    # print(f"\n🔹 Initial clusters (before merging): {len(clusters)}")
    # for i, cluster in enumerate(clusters):
    #     print(f"Cluster {i} (size={len(cluster)}): {cluster}")
    
    print("\nStep 4: Merging similar clusters...")
    merged = merge_clusters_by_similarity(clusters, similarity, threshold=merge_threshold)

    # 🚩 Print clusters after merging but before deduplication
    # print(f"\n🔸 Merged clusters (before deduplication): {len(merged)}")
    # for i, cluster in enumerate(merged):
    #     print(f"Cluster {i} (size={len(cluster)}): {cluster}")
    
    print("\nStep 5: Removing duplicates...")
    final_clusters = remove_duplicates(merged, similarity)

    # 🚩 Print final deduplicated clusters
    print(f"\n✅ Final clusters (after deduplication): {len(final_clusters)}")
    for i, cluster in enumerate(final_clusters):
        print(f"Cluster {i} (size={len(cluster)}): {cluster}")

    # Final cluster densities
    final_densities = [len(cluster) for cluster in final_clusters]
    
    return final_clusters, final_densities, similarity


In [8]:
# Run the revised pipeline
clusters, densities, similarity = run_pipeline(features_df, labels=labels, k=1000, r=0.87, merge_threshold=0.84)


# print("\nFinal clusters after deduplication:")
# # Display the top 15 clusters by size
# for i in range(min(15, len(clusters))):
#     print(f"Cluster {i}: Size = {densities[i]}")
#     print(f"Samples: {clusters[i]}")
#     print("-----------")

# # Also print the total number of samples in all clusters
# total_samples = sum(densities)
# print(f"\nTotal samples in all clusters: {total_samples}")
# print(f"Number of clusters: {len(clusters)}")





Step 2: Computing similarity matrix...
Step 3: Identifying neighbors and densities...

Step 4: Merging similar clusters...
Starting with 624 clusters
Performed 136 recursive similarity-based merges
Found 478 small clusters to potentially merge
Performed 477 additional merges for small clusters
Final number of clusters: 11

Step 5: Removing duplicates...

✅ Final clusters (after deduplication): 11
Cluster 0 (size=139): [30, 65, 85, 93, 94, 98, 100, 113, 115, 116, 119, 124, 144, 153, 155, 160, 161, 167, 176, 177, 178, 182, 184, 186, 192, 193, 194, 195, 206, 210, 221, 223, 225, 237, 242, 246, 248, 249, 259, 265, 273, 276, 297, 349, 362, 422, 423, 430, 431, 433, 434, 437, 438, 440, 442, 455, 469, 470, 472, 473, 474, 475, 476, 477, 478, 484, 485, 486, 487, 489, 490, 491, 492, 494, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 508, 509, 512, 514, 519, 520, 523, 524, 525, 528, 529, 530, 532, 533, 534, 536, 537, 538, 540, 542, 544, 545, 546, 547, 548, 549, 550, 551, 554, 555, 557, 558

# Test to find best parameters

In [ ]:
def explore_parameter_space(features_df, labels=None, k=1000):
    """
    Explore different combinations of r and merge_threshold parameters
    to find optimal settings for clustering.
    
    Args:
        features_df: DataFrame containing features
        labels: Optional array of true labels for evaluation
        k: Number of features to select
        
    Returns:
        best_params: Dictionary with best parameters found
        all_results: DataFrame with all results for further analysis
    """
    import pandas as pd
    import time
    from tqdm import tqdm
    
    # Define parameter ranges to explore
    r_values = [0.95, 0.92, 0.90, 0.87, 0.85, 0.82, 0.80, 0.77, 0.75, 0.72, 0.70, 0.65, 0.60]
    merge_values = [0.80, 0.75, 0.70, 0.65, 0.60, 0.55, 0.50, 0.45, 0.40, 0.35]
    
    # Create combination pairs for more focused exploration
    # We don't need to try all combinations - higher r works better with lower merge threshold
    param_pairs = []
    
    # Add all combinations where r and merge_threshold sum is between 1.3 and 1.7
    # This focuses on the most promising region of parameter space
    for r in r_values:
        for m in merge_values:
            if 1.3 <= r + m <= 1.7:
                param_pairs.append((r, m))
    
    # Add a few extreme combinations to better understand the parameter space
    param_pairs.extend([
        (0.95, 0.80),  # Very strict clustering
        (0.60, 0.40),  # Very loose clustering
        (0.90, 0.40),  # High initial threshold, aggressive merging
        (0.70, 0.80)   # Low initial threshold, conservative merging
    ])
    
    # Remove duplicates
    param_pairs = list(set(param_pairs))
    
    # Prepare results collection
    results = []
    best_cluster_count = float('inf')
    best_params = None
    
    print(f"Exploring {len(param_pairs)} parameter combinations...")
    
    # Run clustering with each parameter combination
    for r, merge_threshold in tqdm(param_pairs):
        start_time = time.time()
        
        # Run the pipeline with current parameters
        clusters, densities, similarity = run_pipeline(
            features_df, 
            labels=labels, 
            k=k, 
            r=r, 
            merge_threshold=merge_threshold,
        )
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Calculate cluster statistics
        num_clusters = len(clusters)
        total_samples = sum(densities)
        
        # Calculate average and median cluster size
        avg_cluster_size = total_samples / num_clusters if num_clusters > 0 else 0
        median_cluster_size = sorted(densities)[len(densities)//2] if densities else 0
        
        # Calculate size of largest and smallest clusters
        largest_cluster = max(densities) if densities else 0
        smallest_cluster = min(densities) if densities else 0
        
        # Calculate distribution stats - how many clusters of different sizes
        small_clusters = sum(1 for size in densities if size <= 3)
        medium_clusters = sum(1 for size in densities if 3 < size <= 10)
        large_clusters = sum(1 for size in densities if size > 10)
        
        # Store results
        result = {
            'r': r,
            'merge_threshold': merge_threshold,
            'num_clusters': num_clusters,
            'total_samples': total_samples,
            'avg_cluster_size': avg_cluster_size,
            'median_cluster_size': median_cluster_size,
            'largest_cluster': largest_cluster,
            'smallest_cluster': smallest_cluster,
            'small_clusters': small_clusters,
            'medium_clusters': medium_clusters,
            'large_clusters': large_clusters,
            'elapsed_time': elapsed_time
        }
        
        results.append(result)
        
        # Check if this is the best parameter set so far
        # We want a reasonable number of clusters (not too many, not too few)
        ideal_cluster_range = (20, 50)  # Adjust this range based on your preferences
        
        # If number of clusters is in the ideal range and better than previous best
        if ideal_cluster_range[0] <= num_clusters <= ideal_cluster_range[1]:
            if abs(num_clusters - sum(ideal_cluster_range)/2) < abs(best_cluster_count - sum(ideal_cluster_range)/2):
                best_cluster_count = num_clusters
                best_params = {'r': r, 'merge_threshold': merge_threshold}
                print(f"New best parameters found: r={r}, merge_threshold={merge_threshold}, clusters={num_clusters}")
        
        # If we haven't found anything in the ideal range, prefer fewer clusters
        elif best_cluster_count > ideal_cluster_range[1] and num_clusters < best_cluster_count:
            best_cluster_count = num_clusters
            best_params = {'r': r, 'merge_threshold': merge_threshold}
            print(f"New best parameters found: r={r}, merge_threshold={merge_threshold}, clusters={num_clusters}")
    
    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(results)
    
    # Sort by number of clusters
    results_df = results_df.sort_values('num_clusters')
    
    # Print summary of best parameter sets
    print("\n===== PARAMETER EXPLORATION RESULTS =====")
    print(f"Best parameters found: r={best_params['r']}, merge_threshold={best_params['merge_threshold']}")
    print(f"Number of clusters with best parameters: {best_cluster_count}")
    
    # Display top 5 parameter combinations with fewest clusters
    print("\nTop 5 parameter combinations with fewest clusters:")
    top_5_fewest = results_df.head(5)
    for _, row in top_5_fewest.iterrows():
        print(f"r={row['r']}, merge_threshold={row['merge_threshold']}: {row['num_clusters']} clusters")
    
    # Display parameter combinations with 20-50 clusters
    ideal_results = results_df[(results_df['num_clusters'] >= 20) & (results_df['num_clusters'] <= 50)]
    if not ideal_results.empty:
        print("\nParameter combinations with 20-50 clusters:")
        for _, row in ideal_results.iterrows():
            print(f"r={row['r']}, merge_threshold={row['merge_threshold']}: {row['num_clusters']} clusters")
    
    # Create a pivot table to visualize the parameter space
    print("\nParameter space visualization (number of clusters):")
    pivot_table = results_df.pivot_table(
        index='r', 
        columns='merge_threshold', 
        values='num_clusters',
        aggfunc='first'  # In case of duplicates
    )
    print(pivot_table)
    
    return best_params, results_df

# Example usage:
best_params, all_results = explore_parameter_space(features_df, labels)

# Use the best parameters found


Exploring 83 parameter combinations...


  0%|          | 0/83 [00:00<?, ?it/s]

Step 2: Computing similarity matrix...
Step 3: Identifying neighbors and densities...

Step 4: Merging similar clusters...
Starting with 624 clusters


  1%|          | 1/83 [00:01<02:19,  1.70s/it]

Performed 613 recursive similarity-based merges
Found 10 small clusters to potentially merge
Performed 7 additional merges for small clusters
Final number of clusters: 4

Step 5: Removing duplicates...

✅ Final clusters (after deduplication): 4
Cluster 0 (size=621): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 1

In [ ]:
print("\Result parameters found:", best_params, "\Result parameters found:", all_results)

\Result parameters found: {'r': 0.9, 'merge_threshold': 0.55} \Result parameters found:        r  merge_threshold  num_clusters  total_samples  avg_cluster_size  \
41  0.60             0.40             3            624        208.000000   
26  0.92             0.40             3            624        208.000000   
46  0.87             0.45             3            624        208.000000   
14  0.90             0.40             3            624        208.000000   
13  0.95             0.40             3            624        208.000000   
..   ...              ...           ...            ...               ...   
29  0.95             0.75             8            624         78.000000   
76  0.85             0.80             8            624         78.000000   
33  0.90             0.75             8            624         78.000000   
51  0.80             0.80            10            624         62.400000   
58  0.77             0.80            11            624         56.727273   


In [ ]:
clusters, densities, similarity = run_pipeline(
    features_df, 
    labels=labels, 
    k=1000, 
    r=best_params['r'],
    merge_threshold=best_params['merge_threshold'],
)

# Visualise all best parameters

In [ ]:
def visualize_clustering_results(clusters, densities, features_df, labels=None, similarity_matrix=None, params=None, all_results=None):
    """
    Comprehensive visualization of clustering results with multiple plots and analyses.
    
    Args:
        clusters: List of clusters (each cluster is a list of sample indices)
        densities: List of cluster sizes
        features_df: DataFrame containing the original features
        labels: Optional ground truth labels for evaluation
        similarity_matrix: Similarity matrix used for clustering
        params: Dictionary with the parameters used (r and merge_threshold)
        all_results: DataFrame with all parameter exploration results (if available)
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import seaborn as sns
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
    from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
    from scipy.spatial.distance import pdist, squareform
    import matplotlib.gridspec as gridspec
    from matplotlib.colors import ListedColormap
    import math
    
    # Set overall style
    plt.style.use('fivethirtyeight')
    
    # Create a color map for clusters
    def get_color_map(n_clusters):
        # Use distinct colors for up to 20 clusters, then recycle
        distinct_colors = plt.cm.tab20(np.linspace(0, 1, 20))
        if n_clusters <= 20:
            return ListedColormap(distinct_colors[:n_clusters])
        else:
            # For more clusters, recycle colors
            return plt.cm.tab20

    # Setup the main figure with multiple subplots
    n_clusters = len(clusters)
    total_samples = sum(densities)
    
    print(f"Visualizing clustering results for {n_clusters} clusters with {total_samples} total samples")
    
    # Assign cluster labels to each sample
    sample_to_cluster = np.zeros(features_df.shape[0], dtype=int) - 1  # Initialize with -1 (no cluster)
    for i, cluster in enumerate(clusters):
        for sample_idx in cluster:
            sample_to_cluster[sample_idx] = i
    
    # Setup subplots layout
    n_plots = 8 if all_results is not None else 6
    fig = plt.figure(figsize=(20, 24))
    gs = gridspec.GridSpec(n_plots, 2, height_ratios=[1, 1, 1, 1.5, 1, 1, 1, 1] if n_plots == 8 else [1, 1, 1, 1.5, 1, 1])
    
    # 1. Cluster Size Distribution
    ax1 = plt.subplot(gs[0, 0])
    sns.histplot(densities, bins=30, kde=True, ax=ax1)
    ax1.set_title("Cluster Size Distribution")
    ax1.set_xlabel("Cluster Size")
    ax1.set_ylabel("Frequency")
    ax1.axvline(np.mean(densities), color='r', linestyle='--', label=f'Mean: {np.mean(densities):.2f}')
    ax1.axvline(np.median(densities), color='g', linestyle='--', label=f'Median: {np.median(densities):.2f}')
    ax1.legend()
    
    # 2. Top Clusters Size Comparison
    top_n = min(20, n_clusters)
    ax2 = plt.subplot(gs[0, 1])
    bars = ax2.bar(range(top_n), densities[:top_n])
    ax2.set_title(f"Size of Top {top_n} Clusters")
    ax2.set_xlabel("Cluster Index")
    ax2.set_ylabel("Number of Samples")
    
    # Add color for bars based on size (darker = larger)
    for i, bar in enumerate(bars):
        bar.set_color(plt.cm.viridis(densities[i]/max(densities)))
    
    # 3. Cluster Size Distribution (cumulative)
    ax3 = plt.subplot(gs[1, 0])
    sorted_sizes = np.sort(densities)[::-1]  # Sort in descending order
    cumulative_sizes = np.cumsum(sorted_sizes)
    cumulative_percentage = cumulative_sizes / total_samples * 100
    
    ax3.plot(range(1, len(cumulative_percentage) + 1), cumulative_percentage, 'b-', marker='o', markersize=3)
    ax3.set_title("Cumulative Sample Distribution")
    ax3.set_xlabel("Number of Clusters")
    ax3.set_ylabel("Cumulative % of Samples")
    
    # Add threshold lines
    for threshold in [50, 80, 90, 95]:
        # Find the number of clusters needed to reach this threshold
        clusters_needed = np.argmax(cumulative_percentage >= threshold) + 1
        ax3.axhline(threshold, color='r', linestyle='--', alpha=0.3)
        ax3.axvline(clusters_needed, color='g', linestyle='--', alpha=0.3)
        ax3.annotate(f"{threshold}% at {clusters_needed} clusters", 
                   xy=(clusters_needed, threshold), 
                   xytext=(clusters_needed + 5, threshold + 2),
                   arrowprops=dict(arrowstyle="->", color="black"))
    
    # 4. Intra-cluster Similarity Heatmap (for top clusters)
    ax4 = plt.subplot(gs[1, 1])
    
    # Calculate average similarity within and between top clusters
    top_k = min(10, n_clusters)  # Show top 10 clusters or fewer
    heat_map_size = top_k
    
    if similarity_matrix is not None:
        similarity_heatmap = np.zeros((heat_map_size, heat_map_size))
        
        # Calculate average similarity within and between clusters
        for i in range(heat_map_size):
            for j in range(heat_map_size):
                cluster_i = clusters[i]
                cluster_j = clusters[j]
                
                # Get all pairwise similarities between samples in the two clusters
                similarities = []
                for idx_i in cluster_i:
                    for idx_j in cluster_j:
                        similarities.append(similarity_matrix[idx_i, idx_j])
                
                # Calculate average similarity
                similarity_heatmap[i, j] = np.mean(similarities) if similarities else 0
        
        # Plot heatmap
        sns.heatmap(similarity_heatmap, annot=True, cmap="YlGnBu", ax=ax4, 
                    vmin=0, vmax=1, fmt='.2f', 
                    xticklabels=range(heat_map_size), 
                    yticklabels=range(heat_map_size))
        ax4.set_title(f"Similarity Between Top {heat_map_size} Clusters")
    else:
        ax4.text(0.5, 0.5, "Similarity matrix not provided", 
                horizontalalignment='center', verticalalignment='center')
    
    # 5. Dimensionality Reduction Visualization (PCA and t-SNE)
    # Convert features to numpy for dimensionality reduction
    X = features_df.values
    
    # First subplot: PCA
    ax5 = plt.subplot(gs[2, 0])
    
    # Apply PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    # Color points by cluster
    scatter = ax5.scatter(X_pca[:, 0], X_pca[:, 1], c=sample_to_cluster, 
                          cmap=get_color_map(n_clusters), 
                          alpha=0.7, s=10)
    ax5.set_title(f"PCA Visualization of {n_clusters} Clusters")
    
    # Add legend for top clusters
    top_n_for_legend = min(10, n_clusters)
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                 markerfacecolor=plt.cm.tab20(i % 20), 
                                 label=f'Cluster {i} (size: {densities[i]})', 
                                 markersize=8) 
                      for i in range(top_n_for_legend)]
    ax5.legend(handles=legend_elements, title="Top Clusters", 
               loc="upper right", bbox_to_anchor=(1.7, 1))
    
    # Second subplot: t-SNE
    ax6 = plt.subplot(gs[2, 1])
    
    # Check if there are enough samples for t-SNE
    max_tsne_samples = 5000  # t-SNE can be slow with large datasets
    if X.shape[0] > max_tsne_samples:
        # Randomly sample a subset
        sample_indices = np.random.choice(X.shape[0], max_tsne_samples, replace=False)
        X_subset = X[sample_indices]
        clusters_subset = sample_to_cluster[sample_indices]
        print(f"Using {max_tsne_samples} random samples for t-SNE visualization")
    else:
        X_subset = X
        clusters_subset = sample_to_cluster
    
    # Apply t-SNE
    try:
        tsne = TSNE(n_components=2, perplexity=min(30, len(X_subset)-1), 
                   n_iter=300, random_state=42)
        X_tsne = tsne.fit_transform(X_subset)
        
        scatter = ax6.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters_subset, 
                              cmap=get_color_map(n_clusters), 
                              alpha=0.7, s=10)
        ax6.set_title(f"t-SNE Visualization of {n_clusters} Clusters")
    except Exception as e:
        ax6.text(0.5, 0.5, f"t-SNE error: {str(e)}", 
                horizontalalignment='center', verticalalignment='center')
    
    # 6. Cluster Quality Metrics
    ax7 = plt.subplot(gs[3, :])
    
    # Create a DataFrame for cluster quality metrics
    metrics_df = pd.DataFrame({
        'Cluster': range(n_clusters),
        'Size': densities,
    })
    
    # Add average intra-cluster similarity
    if similarity_matrix is not None:
        intra_similarities = []
        for i, cluster in enumerate(clusters):
            if len(cluster) > 1:
                # Get all pairwise similarities within the cluster
                cluster_sim = 0
                count = 0
                for idx_i in range(len(cluster)):
                    for idx_j in range(idx_i+1, len(cluster)):
                        cluster_sim += similarity_matrix[cluster[idx_i], cluster[idx_j]]
                        count += 1
                        
                avg_sim = cluster_sim / count if count > 0 else 0
            else:
                avg_sim = 1.0  # Single-element cluster
                
            intra_similarities.append(avg_sim)
            
        metrics_df['Intra-Similarity'] = intra_similarities
    
    # Calculate cluster quality metrics where possible
    try:
        if len(np.unique(sample_to_cluster[sample_to_cluster >= 0])) > 1 and similarity_matrix is not None:
            # Get only samples that are in clusters
            clustered_indices = np.where(sample_to_cluster >= 0)[0]
            clustered_features = X[clustered_indices]
            clustered_labels = sample_to_cluster[clustered_indices]
            
            # Calculate silhouette score
            silhouette = silhouette_score(1 - similarity_matrix[clustered_indices][:, clustered_indices], 
                                          clustered_labels, metric='precomputed')
            metrics_df['Silhouette Score'] = silhouette
            
            # Calculate other metrics if we have enough samples
            try:
                ch_score = calinski_harabasz_score(clustered_features, clustered_labels)
                db_score = davies_bouldin_score(clustered_features, clustered_labels)
                metrics_df['CH Score'] = ch_score
                metrics_df['DB Score'] = db_score
            except Exception as e:
                print(f"Error calculating cluster validity metrics: {e}")
    except Exception as e:
        print(f"Error calculating silhouette score: {e}")
    
    # If ground truth labels are provided, calculate external metrics
    if labels is not None:
        try:
            # Calculate ARI and NMI only for samples assigned to clusters
            clustered_indices = np.where(sample_to_cluster >= 0)[0]
            true_labels = labels[clustered_indices]
            pred_labels = sample_to_cluster[clustered_indices]
            
            # Calculate external validation metrics
            if len(np.unique(true_labels)) > 1:
                ari = adjusted_rand_score(true_labels, pred_labels)
                nmi = normalized_mutual_info_score(true_labels, pred_labels)
                metrics_df['ARI'] = ari
                metrics_df['NMI'] = nmi
        except Exception as e:
            print(f"Error calculating external validation metrics: {e}")
    
    # Plot metrics as a horizontal bar chart
    metric_columns = [col for col in metrics_df.columns if col != 'Cluster' and col != 'Size']
    
    if metric_columns:
        # Fixed the error with metrics table: Convert explicitly to strings for table cells
        metrics_mean = metrics_df[metric_columns].mean().to_frame().T
        metrics_mean['Metric'] = 'Average'
        
        # Convert the metrics to string with formatting applied
        cell_text = []
        for _, row in metrics_mean.iterrows():
            row_text = ['Average']  # Start with the row label
            for col in metric_columns:
                # Format numeric values to 3 decimal places
                if isinstance(row[col], (int, float)):
                    row_text.append(f"{row[col]:.3f}")
                else:
                    row_text.append(str(row[col]))
            cell_text.append(row_text)
        
        metrics_table = ax7.table(
            cellText=cell_text,
            colLabels=['Metric'] + metric_columns,
            loc='center',
            cellLoc='center'
        )
        metrics_table.auto_set_font_size(False)
        metrics_table.set_fontsize(12)
        metrics_table.scale(1, 2)
        ax7.axis('off')
        ax7.set_title("Clustering Quality Metrics", pad=20)
    else:
        ax7.text(0.5, 0.5, "No metrics calculated", 
                horizontalalignment='center', verticalalignment='center')
    
    # 7. Parameter Investigation (if parameter exploration results are available)
    if all_results is not None:
        # Plot parameter heatmap
        ax8 = plt.subplot(gs[4, :])
        
        # Create pivot table for parameter exploration
        try:
            pivot = all_results.pivot_table(
                index='r', 
                columns='merge_threshold', 
                values='num_clusters',
                aggfunc='first'
            )
            
            # Plot heatmap
            sns.heatmap(pivot, annot=True, cmap="YlGnBu", ax=ax8, fmt='g')
            ax8.set_title("Number of Clusters by Parameter Combination")
            
            # Highlight the current parameters if provided
            if params and 'r' in params and 'merge_threshold' in params:
                r_idx = np.where(pivot.index == params['r'])[0]
                m_idx = np.where(pivot.columns == params['merge_threshold'])[0]
                
                if len(r_idx) > 0 and len(m_idx) > 0:
                    r_pos = r_idx[0]
                    m_pos = m_idx[0]
                    ax8.add_patch(plt.Rectangle((m_pos, r_pos), 1, 1, fill=False, 
                                             edgecolor='red', lw=3))
        except Exception as e:
            ax8.text(0.5, 0.5, f"Error creating parameter heatmap: {str(e)}", 
                    horizontalalignment='center', verticalalignment='center')
        
        # Plot parameter impact on cluster count
        ax9 = plt.subplot(gs[5, 0])
        ax10 = plt.subplot(gs[5, 1])
        
        # Effect of r on number of clusters
        r_impact = all_results.groupby('r')['num_clusters'].mean().reset_index()
        sns.lineplot(data=r_impact, x='r', y='num_clusters', ax=ax9, marker='o')
        ax9.set_title("Effect of r on Cluster Count")
        ax9.set_xlabel("r (similarity threshold)")
        ax9.set_ylabel("Average Number of Clusters")
        
        # Effect of merge_threshold on number of clusters
        m_impact = all_results.groupby('merge_threshold')['num_clusters'].mean().reset_index()
        sns.lineplot(data=m_impact, x='merge_threshold', y='num_clusters', ax=ax10, marker='o')
        ax10.set_title("Effect of merge_threshold on Cluster Count")
        ax10.set_xlabel("merge_threshold")
        ax10.set_ylabel("Average Number of Clusters")
        
        # Additional visualizations for parameter exploration
        ax11 = plt.subplot(gs[6, 0])
        ax12 = plt.subplot(gs[6, 1])
        
        # Scatterplot of parameters colored by cluster count
        scatter = ax11.scatter(all_results['r'], all_results['merge_threshold'], 
                            c=all_results['num_clusters'], cmap='viridis', 
                            s=100, alpha=0.7)
        if params and 'r' in params and 'merge_threshold' in params:
            ax11.scatter([params['r']], [params['merge_threshold']], 
                       color='red', s=200, marker='*', 
                       label=f"Current (r={params['r']}, m={params['merge_threshold']})")
            ax11.legend()
            
        plt.colorbar(scatter, ax=ax11, label="Number of Clusters")
        ax11.set_title("Parameter Space Exploration")
        ax11.set_xlabel("r (similarity threshold)")
        ax11.set_ylabel("merge_threshold")
        
        # Plot distribution of cluster counts
        sns.histplot(all_results['num_clusters'], kde=True, ax=ax12)
        ax12.set_title("Distribution of Cluster Counts")
        ax12.set_xlabel("Number of Clusters")
        ax12.set_ylabel("Frequency")
        
        if params and 'r' in params and 'merge_threshold' in params:
            current_clusters = n_clusters
            ax12.axvline(current_clusters, color='r', linestyle='--', 
                       label=f"Current: {current_clusters}")
            ax12.legend()
        
        # Final parameter summary - fixed table formatting
        ax13 = plt.subplot(gs[7, :])
        
        # Create a summary table of best parameter combinations
        best_params = all_results.sort_values('num_clusters').head(10)
        best_params = best_params[['r', 'merge_threshold', 'num_clusters', 
                                 'avg_cluster_size', 'largest_cluster']]
        
        # Format the table with strings
        cell_text = []
        for _, row in best_params.iterrows():
            cell_text.append([
                f"{row['r']:.2f}",
                f"{row['merge_threshold']:.2f}",
                f"{int(row['num_clusters'])}",
                f"{row['avg_cluster_size']:.1f}",
                f"{int(row['largest_cluster'])}"
            ])
            
        param_table = ax13.table(
            cellText=cell_text,
            colLabels=['r', 'merge_threshold', 'num_clusters', 'avg_size', 'largest_cluster'],
            loc='center',
            cellLoc='center'
        )
        param_table.auto_set_font_size(False)
        param_table.set_fontsize(12)
        param_table.scale(1, 2)
        ax13.axis('off')
        ax13.set_title("Top 10 Parameter Combinations with Fewest Clusters", pad=20)
    
    # Make layout tight and show the plot
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    
    return fig

# Example usage:
fig = visualize_clustering_results(
    clusters=clusters, 
    densities=densities,
    features_df=features_df, 
    labels=labels,  # Optional
    similarity_matrix=similarity,  # Pass the similarity matrix
    params=best_params,  # Pass the parameters used
    all_results=all_results  # Pass parameter exploration results
)
plt.savefig('clustering_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

NameError: name 'clusters' is not defined

In [ ]:
def visualize_clustering_detailed(clusters, densities, features_df, labels=None, similarity_matrix=None, params=None, all_results=None, output_dir="cluster_visualizations"):
    """
    Creates separate high-quality visualizations for each aspect of clustering results
    
    Args:
        clusters: List of clusters (each cluster is a list of sample indices)
        densities: List of cluster sizes
        features_df: DataFrame containing the original features
        labels: Optional ground truth labels for evaluation
        similarity_matrix: Similarity matrix used for clustering
        params: Dictionary with the parameters used (r and merge_threshold)
        all_results: DataFrame with all parameter exploration results (if available)
        output_dir: Directory to save visualizations (will be created if it doesn't exist)
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import seaborn as sns
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
    from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
    import matplotlib.gridspec as gridspec
    from matplotlib.colors import ListedColormap
    import os
    from matplotlib.cm import get_cmap
    import matplotlib.ticker as ticker
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    
    # Set overall style
    plt.style.use('fivethirtyeight')
    sns.set_style("whitegrid")
    
    # Basic statistics
    n_clusters = len(clusters)
    total_samples = sum(densities)
    print(f"Visualizing clustering results for {n_clusters} clusters with {total_samples} total samples")
    
    # Assign cluster labels to each sample
    sample_to_cluster = np.zeros(features_df.shape[0], dtype=int) - 1  # Initialize with -1 (no cluster)
    for i, cluster in enumerate(clusters):
        for sample_idx in cluster:
            sample_to_cluster[sample_idx] = i
    
    # Helper function to create color map
    def get_color_map(n_clusters):
        # Use distinct colors for up to 20 clusters, then recycle
        colors = plt.cm.tab20(np.linspace(0, 1, 20))
        return colors[:min(n_clusters, 20)]
    
    # 1. Cluster Size Distribution
    plt.figure(figsize=(12, 8))
    ax = plt.gca()
    
    # Create custom bins to handle different distributions
    max_cluster_size = max(densities)
    if max_cluster_size > 100:
        bins = np.linspace(0, max_cluster_size, 30)
    else:
        bins = 20
        
    sns.histplot(densities, bins=bins, kde=True, ax=ax, color='royalblue', alpha=0.7)
    ax.set_title("Cluster Size Distribution", fontsize=16)
    ax.set_xlabel("Cluster Size", fontsize=14)
    ax.set_ylabel("Frequency", fontsize=14)
    
    # Add mean and median lines
    mean_size = np.mean(densities)
    median_size = np.median(densities)
    ax.axvline(mean_size, color='darkred', linestyle='--', linewidth=2, 
               label=f'Mean: {mean_size:.2f}')
    ax.axvline(median_size, color='darkgreen', linestyle='--', linewidth=2, 
               label=f'Median: {median_size:.2f}')
    
    ax.legend(fontsize=12, loc='upper right')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/1_cluster_size_distribution.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Top Clusters Size Comparison
    # Create a custom chart showing the top 5 clusters with details
    top_n = 5  # Show top 5 clusters
    
    # Create a DataFrame with cluster information
    top_clusters_df = pd.DataFrame({
        'Cluster': range(n_clusters),
        'Size': densities
    }).sort_values('Size', ascending=False).head(top_n)
    
    plt.figure(figsize=(14, 8))
    ax = plt.gca()
    
    # Use a specific colormap
    colors = plt.cm.viridis(np.linspace(0.1, 0.9, top_n))
    
    bars = ax.bar(
        top_clusters_df['Cluster'], 
        top_clusters_df['Size'],
        color=colors,
        width=0.6
    )
    
    # Add values on top of bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width()/2.,
            height + 5,
            f'{int(height)}',
            ha='center', 
            va='bottom',
            fontsize=12,
            fontweight='bold'
        )
    
    ax.set_title(f"Top {top_n} Largest Clusters", fontsize=16)
    ax.set_xlabel("Cluster ID", fontsize=14)
    ax.set_ylabel("Number of Samples", fontsize=14)
    
    # Add percentage of total samples for each top cluster
    for i, (_, row) in enumerate(top_clusters_df.iterrows()):
        percent = (row['Size'] / total_samples) * 100
        ax.annotate(
            f"{percent:.1f}% of total",
            xy=(row['Cluster'], row['Size'] / 2),
            ha='center',
            va='center',
            fontsize=11,
            color='white',
            fontweight='bold'
        )
    
    # Add total percentage covered by top N clusters
    top_n_percent = (top_clusters_df['Size'].sum() / total_samples) * 100
    ax.text(
        0.5, 0.01,
        f"These {top_n} clusters contain {top_n_percent:.1f}% of all samples",
        transform=ax.transAxes,
        ha='center',
        fontsize=13,
        fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.5')
    )
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/2_top_{top_n}_clusters.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Cumulative Sample Distribution
    plt.figure(figsize=(12, 8))
    ax = plt.gca()
    
    sorted_sizes = np.sort(densities)[::-1]  # Sort in descending order
    cumulative_sizes = np.cumsum(sorted_sizes)
    cumulative_percentage = cumulative_sizes / total_samples * 100
    
    # Plot cumulative distribution
    ax.plot(
        range(1, len(cumulative_percentage) + 1), 
        cumulative_percentage, 
        'b-', 
        marker='o', 
        markersize=5,
        linewidth=2,
        color='royalblue'
    )
    
    ax.set_title("Cumulative Sample Distribution", fontsize=16)
    ax.set_xlabel("Number of Clusters", fontsize=14)
    ax.set_ylabel("Cumulative % of Samples", fontsize=14)
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Add threshold lines with better visibility
    for threshold in [50, 80, 90, 95]:
        # Find the number of clusters needed to reach this threshold
        clusters_needed = np.argmax(cumulative_percentage >= threshold) + 1
        
        # Only add if we have enough clusters to reach this threshold
        if clusters_needed <= len(cumulative_percentage):
            # Add horizontal line
            ax.axhline(
                threshold, 
                color='crimson', 
                linestyle='--', 
                alpha=0.5,
                linewidth=1.5
            )
            
            # Add vertical line
            ax.axvline(
                clusters_needed, 
                color='darkgreen', 
                linestyle='--', 
                alpha=0.5,
                linewidth=1.5
            )
            
            # Add annotation with arrow
            ax.annotate(
                f"{threshold}% at {clusters_needed} clusters", 
                xy=(clusters_needed, threshold), 
                xytext=(clusters_needed + max(1, n_clusters/20), threshold + 5),
                arrowprops=dict(
                    arrowstyle="->", 
                    color="black",
                    linewidth=1.5
                ),
                fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8)
            )
    
    # Set y-axis limits
    ax.set_ylim(0, 105)
    
    # Set x-axis limit to show a bit more than we need
    ax.set_xlim(0, min(n_clusters * 1.1, n_clusters + 5))
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/3_cumulative_distribution.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 4. Intra-cluster Similarity Heatmap (for top clusters)
    if similarity_matrix is not None:
        plt.figure(figsize=(14, 12))
        ax = plt.gca()
        
        # Calculate average similarity within and between top clusters
        top_k = min(10, n_clusters)  # Show top 10 clusters or fewer
        top_clusters = np.argsort(densities)[::-1][:top_k]
        
        similarity_heatmap = np.zeros((top_k, top_k))
        
        # Calculate average similarity within and between clusters
        for i, cluster_i_idx in enumerate(top_clusters):
            for j, cluster_j_idx in enumerate(top_clusters):
                cluster_i = clusters[cluster_i_idx]
                cluster_j = clusters[cluster_j_idx]
                
                # Get all pairwise similarities between samples in the two clusters
                similarities = []
                for idx_i in cluster_i:
                    for idx_j in cluster_j:
                        similarities.append(similarity_matrix[idx_i, idx_j])
                
                # Calculate average similarity
                similarity_heatmap[i, j] = np.mean(similarities) if similarities else 0
        
        # Create custom labels showing cluster index and size
        labels = [f"{idx}\n(size: {densities[idx]})" for idx in top_clusters]
        
        # Plot heatmap with improved aesthetics
        sns.heatmap(
            similarity_heatmap, 
            annot=True, 
            cmap="YlGnBu", 
            ax=ax, 
            vmin=0, 
            vmax=1, 
            fmt='.2f', 
            xticklabels=labels, 
            yticklabels=labels,
            linewidths=0.5,
            annot_kws={"size": 10, "weight": "bold"}
        )
        
        # Add titles and labels
        ax.set_title(f"Similarity Between Top {top_k} Clusters", fontsize=16)
        ax.set_xlabel("Cluster ID (size)", fontsize=14)
        ax.set_ylabel("Cluster ID (size)", fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/4_similarity_heatmap.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # 5. Dimensionality Reduction Visualization (PCA)
    # Convert features to numpy for dimensionality reduction
    X = features_df.values
    
    # Apply PCA
    plt.figure(figsize=(12, 10))
    ax = plt.gca()
    
    # Apply PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    # Get a colormap
    colors = get_cmap('tab20')
    
    # Plot the top 10 largest clusters with different colors
    top_clusters = np.argsort(densities)[::-1][:10]
    
    legend_elements = []
    
    # Plot unclustered points first (if any)
    unclustered = np.where(sample_to_cluster == -1)[0]
    if len(unclustered) > 0:
        ax.scatter(
            X_pca[unclustered, 0], 
            X_pca[unclustered, 1], 
            c='lightgray', 
            alpha=0.5, 
            s=20,
            label="Unclustered"
        )
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                      markerfacecolor='lightgray', 
                                      label=f'Unclustered ({len(unclustered)})', 
                                      markersize=10))
    
    # Plot each top cluster
    for i, cluster_idx in enumerate(top_clusters):
        cluster_samples = np.where(sample_to_cluster == cluster_idx)[0]
        ax.scatter(
            X_pca[cluster_samples, 0],
            X_pca[cluster_samples, 1],
            c=[colors(i % 20)],
            alpha=0.7,
            s=30,
            label=f"Cluster {cluster_idx} (size: {densities[cluster_idx]})"
        )
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                      markerfacecolor=colors(i % 20), 
                                      label=f'Cluster {cluster_idx} (size: {densities[cluster_idx]})', 
                                      markersize=10))
    
    # Add variance explained
    var_explained = pca.explained_variance_ratio_
    ax.set_xlabel(f"PC1 ({var_explained[0]:.2%} variance)", fontsize=14)
    ax.set_ylabel(f"PC2 ({var_explained[1]:.2%} variance)", fontsize=14)
    
    ax.set_title(f"PCA Visualization of Top Clusters", fontsize=16)
    
    # Add legend with better positioning
    ax.legend(
        handles=legend_elements, 
        title="Clusters", 
        loc="upper right",
        bbox_to_anchor=(1.15, 1),
        fontsize=12
    )
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/5_pca_visualization.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 6. t-SNE Visualization
    plt.figure(figsize=(12, 10))
    ax = plt.gca()
    
    # Check if there are enough samples for t-SNE
    max_tsne_samples = 5000  # t-SNE can be slow with large datasets
    if X.shape[0] > max_tsne_samples:
        # Randomly sample a subset
        np.random.seed(42)  # for reproducibility
        sample_indices = np.random.choice(X.shape[0], max_tsne_samples, replace=False)
        X_subset = X[sample_indices]
        clusters_subset = sample_to_cluster[sample_indices]
        print(f"Using {max_tsne_samples} random samples for t-SNE visualization")
    else:
        X_subset = X
        clusters_subset = sample_to_cluster
    
    # Apply t-SNE with error handling
    try:
        tsne = TSNE(
            n_components=2, 
            perplexity=min(30, len(X_subset)-1), 
            n_iter=1000,  # Increased for better convergence
            random_state=42
        )
        X_tsne = tsne.fit_transform(X_subset)
        
        # Plot unclustered points first (if any)
        unclustered = np.where(clusters_subset == -1)[0]
        if len(unclustered) > 0:
            ax.scatter(
                X_tsne[unclustered, 0], 
                X_tsne[unclustered, 1], 
                c='lightgray', 
                alpha=0.5, 
                s=20,
                label="Unclustered"
            )
            legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                          markerfacecolor='lightgray', 
                                          label=f'Unclustered', 
                                          markersize=10))
        
        # Plot each top cluster
        legend_elements = []
        for i, cluster_idx in enumerate(top_clusters):
            cluster_samples = np.where(clusters_subset == cluster_idx)[0]
            if len(cluster_samples) > 0:
                ax.scatter(
                    X_tsne[cluster_samples, 0],
                    X_tsne[cluster_samples, 1],
                    c=[colors(i % 20)],
                    alpha=0.7,
                    s=30,
                    label=f"Cluster {cluster_idx} (size: {densities[cluster_idx]})"
                )
                legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                              markerfacecolor=colors(i % 20), 
                                              label=f'Cluster {cluster_idx}', 
                                              markersize=10))
        
        ax.set_title(f"t-SNE Visualization of Top Clusters", fontsize=16)
        
        # Add legend with better positioning
        ax.legend(
            handles=legend_elements, 
            title="Clusters", 
            loc="upper right",
            bbox_to_anchor=(1.15, 1),
            fontsize=12
        )
        
    except Exception as e:
        ax.text(0.5, 0.5, f"t-SNE error: {str(e)}", 
               horizontalalignment='center', verticalalignment='center',
               fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/6_tsne_visualization.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 7. Cluster Quality Metrics Table
    # Create a DataFrame for cluster quality metrics
    metrics_df = pd.DataFrame({
        'Cluster': range(n_clusters),
        'Size': densities,
    })
    
    # Add average intra-cluster similarity
    if similarity_matrix is not None:
        intra_similarities = []
        for i, cluster in enumerate(clusters):
            if len(cluster) > 1:
                # Get all pairwise similarities within the cluster
                cluster_sim = 0
                count = 0
                for idx_i in range(len(cluster)):
                    for idx_j in range(idx_i+1, len(cluster)):
                        cluster_sim += similarity_matrix[cluster[idx_i], cluster[idx_j]]
                        count += 1
                        
                avg_sim = cluster_sim / count if count > 0 else 0
            else:
                avg_sim = 1.0  # Single-element cluster
                
            intra_similarities.append(avg_sim)
            
        metrics_df['Intra-Similarity'] = intra_similarities
    
    # Calculate cluster quality metrics where possible
    metrics_values = {}
    
    try:
        if len(np.unique(sample_to_cluster[sample_to_cluster >= 0])) > 1 and similarity_matrix is not None:
            # Get only samples that are in clusters
            clustered_indices = np.where(sample_to_cluster >= 0)[0]
            clustered_features = X[clustered_indices]
            clustered_labels = sample_to_cluster[clustered_indices]
            
            # Calculate silhouette score
            silhouette = silhouette_score(1 - similarity_matrix[clustered_indices][:, clustered_indices], 
                                         clustered_labels, metric='precomputed')
            metrics_values['Silhouette Score'] = silhouette
            
            # Calculate other metrics if we have enough samples
            try:
                ch_score = calinski_harabasz_score(clustered_features, clustered_labels)
                db_score = davies_bouldin_score(clustered_features, clustered_labels)
                metrics_values['CH Score'] = ch_score
                metrics_values['DB Score'] = db_score
            except Exception as e:
                print(f"Error calculating cluster validity metrics: {e}")
    except Exception as e:
        print(f"Error calculating silhouette score: {e}")
    
    # If ground truth labels are provided, calculate external metrics
    if labels is not None:
        try:
            # Calculate ARI and NMI only for samples assigned to clusters
            clustered_indices = np.where(sample_to_cluster >= 0)[0]
            true_labels = labels[clustered_indices]
            pred_labels = sample_to_cluster[clustered_indices]
            
            # Calculate external validation metrics
            if len(np.unique(true_labels)) > 1:
                ari = adjusted_rand_score(true_labels, pred_labels)
                nmi = normalized_mutual_info_score(true_labels, pred_labels)
                metrics_values['ARI'] = ari
                metrics_values['NMI'] = nmi
        except Exception as e:
            print(f"Error calculating external validation metrics: {e}")
    
    # Create a detailed metrics visualization
    plt.figure(figsize=(14, 6))
    ax = plt.gca()
    ax.axis('tight')
    ax.axis('off')
    
    # Prepare data for the table
    if metrics_values:
        # Format the metrics values
        formatted_values = []
        for key, value in metrics_values.items():
            if isinstance(value, (int, float)):
                formatted_values.append(f"{value:.4f}")
            else:
                formatted_values.append(str(value))
        
        metrics_table = ax.table(
            cellText=[formatted_values],
            colLabels=list(metrics_values.keys()),
            loc='center',
            cellLoc='center'
        )
        
        # Style the table
        metrics_table.auto_set_font_size(False)
        metrics_table.set_fontsize(12)
        metrics_table.scale(1, 2)
        
        # Add title
        plt.suptitle("Clustering Quality Metrics", fontsize=16, y=0.8)
        
        # Add description of metrics
        descriptions = {
            'Silhouette Score': "Measures how similar a point is to its own cluster compared to others. Range: [-1, 1], higher is better.",
            'CH Score': "Calinski-Harabasz index - ratio of between-cluster to within-cluster dispersion. Higher is better.",
            'DB Score': "Davies-Bouldin index - average similarity of each cluster with its most similar cluster. Lower is better.",
            'ARI': "Adjusted Rand Index - similarity between true and predicted labels, adjusted for chance. Range: [-1, 1], higher is better.",
            'NMI': "Normalized Mutual Information - normalized measure of dependence between true and predicted labels. Range: [0, 1], higher is better."
        }
        
        desc_text = ""
        for key in metrics_values.keys():
            if key in descriptions:
                desc_text += f"• {key}: {descriptions[key]}\n"
        
        plt.figtext(0.5, 0.3, desc_text, ha='center', fontsize=10, 
                   bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
    else:
        ax.text(0.5, 0.5, "No metrics calculated", 
                ha='center', va='center', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/7_cluster_quality_metrics.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 8. Top 5 Clusters Detailed Analysis
    # For a detailed look at the top 5 clusters
    top_5_clusters = np.argsort(densities)[::-1][:5]
    
    plt.figure(figsize=(14, 10))
    
    # Create a subplot for each top cluster's detailed analysis
    for i, cluster_idx in enumerate(top_5_clusters):
        ax = plt.subplot(3, 2, i+1)
        
        # Get the cluster members
        cluster = clusters[cluster_idx]
        cluster_size = len(cluster)
        
        # Calculate intra-cluster statistics
        if similarity_matrix is not None:
            intra_similarities = []
            for idx_i in range(len(cluster)):
                for idx_j in range(idx_i+1, len(cluster)):
                    intra_similarities.append(similarity_matrix[cluster[idx_i], cluster[idx_j]])
            
            mean_sim = np.mean(intra_similarities) if intra_similarities else 0
            min_sim = np.min(intra_similarities) if intra_similarities else 0
            max_sim = np.max(intra_similarities) if intra_similarities else 0
            
            # Create a histogram of intra-cluster similarities
            if intra_similarities:
                sns.histplot(intra_similarities, kde=True, ax=ax, color=colors(i % 20), alpha=0.7)
                
                # Add vertical lines for mean, min, max
                ax.axvline(mean_sim, color='red', linestyle='--', 
                          label=f'Mean: {mean_sim:.3f}')
                ax.axvline(min_sim, color='blue', linestyle=':', 
                          label=f'Min: {min_sim:.3f}')
                ax.axvline(max_sim, color='green', linestyle=':', 
                          label=f'Max: {max_sim:.3f}')
                
                ax.set_title(f"Cluster {cluster_idx} (Size: {cluster_size})", fontsize=14)
                ax.set_xlabel("Intra-cluster Similarity", fontsize=12)
                ax.set_ylabel("Frequency", fontsize=12)
                ax.legend(fontsize=10)
        else:
            ax.text(0.5, 0.5, "Similarity matrix not provided", 
                   ha='center', va='center', fontsize=12)
    
    plt.tight_layout()
    plt.suptitle("Detailed Analysis of Top 5 Clusters", fontsize=16, y=1.02)
    plt.savefig(f"{output_dir}/8_top_5_clusters_analysis.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 9. Parameter Space Visualization (if parameter exploration results are available)
    if all_results is not None:
        # 9.1 Parameter Heatmap
        plt.figure(figsize=(14, 10))
        ax = plt.gca()
        
        # Create pivot table for parameter exploration
        try:
            pivot = all_results.pivot_table(
                index='r', 
                columns='merge_threshold', 
                values='num_clusters',
                aggfunc='first'
            )
            
            # Sort indices for better visualization
            pivot = pivot.sort_index(ascending=False)
            pivot = pivot.sort_index(axis=1)
            
            # Plot heatmap
            sns.heatmap(
                pivot, 
                annot=True, 
                cmap="YlGnBu", 
                ax=ax, 
                fmt='g',
                linewidths=0.5,
                annot_kws={"size": 10, "weight": "bold"}
            )
            
            ax.set_title("Number of Clusters by Parameter Combination", fontsize=16)
            ax.set_xlabel("merge_threshold", fontsize=14)
            ax.set_ylabel("r (similarity threshold)", fontsize=14)
            
            # Highlight the current parameters if provided
            if params and 'r' in params and 'merge_threshold' in params:
                try:
                    r_idx = np.where(pivot.index == params['r'])[0]
                    m_idx = np.where(pivot.columns == params['merge_threshold'])[0]
                    
                    if len(r_idx) > 0 and len(m_idx) > 0:
                        r_pos = r_idx[0]
                        m_pos = m_idx[0]
                        ax.add_patch(plt.Rectangle((m_pos, r_pos), 1, 1, fill=False, 
                                                 edgecolor='red', lw=3))
                        
                        # Add text annotation
                        plt.text(
                            0.5, 0.01,
                            f"Current parameters: r={params['r']}, merge_threshold={params['merge_threshold']}",
                            transform=ax.transAxes,
                            ha='center',
                            fontsize=12,
                            fontweight='bold',
                            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.5')
                        )
                except Exception as e:
                    print(f"Error highlighting current parameters: {e}")
                    
        except Exception as e:
            ax.text(0.5, 0.5, f"Error creating parameter heatmap: {str(e)}", 
                   ha='center', va='center', fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/9_parameter_heatmap.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        # 9.2 Parameter Impact on Cluster Count
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
        
        # Effect of r on number of clusters
        r_impact = all_results.groupby('r')['num_clusters'].mean().reset_index()
        r_impact = r_impact.sort_values('r')
        
        sns.lineplot(data=r_impact, x='r', y='num_clusters', ax=ax1, 
                    marker='o', linewidth=2, markersize=8, color='royalblue')
        ax1.set_title("Effect of r on Cluster Count", fontsize=16)
        ax1.set_xlabel("r (similarity threshold)", fontsize=14)
        ax1.set_ylabel("Average Number of Clusters", fontsize=14)
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # Add current r parameter if available
        if params and 'r' in params:
            ax1.axvline(params['r'], color='red', linestyle='--', 
                      label=f"Current r={params['r']}")
            ax1.legend(fontsize=12)

# Effect of merge_threshold on number of clusters
        m_impact = all_results.groupby('merge_threshold')['num_clusters'].mean().reset_index()
        m_impact = m_impact.sort_values('merge_threshold')
        
        sns.lineplot(data=m_impact, x='merge_threshold', y='num_clusters', ax=ax2, 
                    marker='o', linewidth=2, markersize=8, color='forestgreen')
        ax2.set_title("Effect of merge_threshold on Cluster Count", fontsize=16)
        ax2.set_xlabel("merge_threshold", fontsize=14)
        ax2.set_ylabel("Average Number of Clusters", fontsize=14)
        ax2.grid(True, linestyle='--', alpha=0.7)
        
        # Add current merge_threshold parameter if available
        if params and 'merge_threshold' in params:
            ax2.axvline(params['merge_threshold'], color='red', linestyle='--', 
                      label=f"Current m={params['merge_threshold']}")
            ax2.legend(fontsize=12)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/10_parameter_impact.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        # 9.3 Parameter Space Exploration
        plt.figure(figsize=(12, 10))
        ax = plt.gca()
        
        # Scatterplot of parameters colored by cluster count
        scatter = ax.scatter(all_results['r'], all_results['merge_threshold'], 
                           c=all_results['num_clusters'], cmap='viridis', 
                           s=150, alpha=0.8, edgecolor='black')
        
        # Add current parameters if provided
        if params and 'r' in params and 'merge_threshold' in params:
            ax.scatter([params['r']], [params['merge_threshold']], 
                     color='red', s=250, marker='*', edgecolor='black', linewidth=2,
                     label=f"Current (r={params['r']}, m={params['merge_threshold']})")
            ax.legend(fontsize=12, loc='upper right')
            
        plt.colorbar(scatter, ax=ax, label="Number of Clusters")
        ax.set_title("Parameter Space Exploration", fontsize=16)
        ax.set_xlabel("r (similarity threshold)", fontsize=14)
        ax.set_ylabel("merge_threshold", fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.5)
        
        # Add text annotations for interesting points
        # Find min and max cluster count parameters
        min_cluster_row = all_results.loc[all_results['num_clusters'].idxmin()]
        max_cluster_row = all_results.loc[all_results['num_clusters'].idxmax()]
        
        # Add annotations
        ax.annotate(
            f"Min clusters: {int(min_cluster_row['num_clusters'])}",
            xy=(min_cluster_row['r'], min_cluster_row['merge_threshold']),
            xytext=(min_cluster_row['r'] - 0.05, min_cluster_row['merge_threshold'] - 0.05),
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="black"),
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
            fontsize=10
        )
        
        ax.annotate(
            f"Max clusters: {int(max_cluster_row['num_clusters'])}",
            xy=(max_cluster_row['r'], max_cluster_row['merge_threshold']),
            xytext=(max_cluster_row['r'] + 0.05, max_cluster_row['merge_threshold'] + 0.05),
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="black"),
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
            fontsize=10
        )
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/11_parameter_space.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        # 10. Best Parameter Combinations Table
        plt.figure(figsize=(14, 8))
        ax = plt.gca()
        ax.axis('tight')
        ax.axis('off')
        
        # Create a summary table of best parameter combinations
        best_params_df = all_results.sort_values('num_clusters').head(10)
        best_params_df = best_params_df[['r', 'merge_threshold', 'num_clusters', 
                                     'avg_cluster_size', 'largest_cluster']]
        
        # Format the table with strings
        cell_text = []
        for _, row in best_params_df.iterrows():
            cell_text.append([
                f"{row['r']:.2f}",
                f"{row['merge_threshold']:.2f}",
                f"{int(row['num_clusters'])}",
                f"{row['avg_cluster_size']:.1f}",
                f"{int(row['largest_cluster'])}"
            ])
            
        param_table = ax.table(
            cellText=cell_text,
            colLabels=['r', 'merge_threshold', 'num_clusters', 'avg_size', 'largest_cluster'],
            loc='center',
            cellLoc='center'
        )
        param_table.auto_set_font_size(False)
        param_table.set_fontsize(12)
        param_table.scale(1, 2)
        
        # Highlight the best parameter row
        for i in range(len(cell_text[0])):
            param_table[1, i].set_facecolor('#aaffaa')
            param_table[1, i].set_text_props(weight='bold')
        
        plt.suptitle("Top 10 Parameter Combinations with Fewest Clusters", fontsize=16, y=0.9)
        
        # Add a recommendation text
        recommend_text = (
            "Recommendation: Choose parameters that balance the number of clusters and cluster quality.\n"
            "Fewer clusters with meaningful sizes are generally preferable for easier interpretation.\n"
            "The highlighted row represents the parameter combination with fewest clusters."
        )
        plt.figtext(0.5, 0.15, recommend_text, ha='center', fontsize=12, 
                   bbox=dict(facecolor='#f0f0f0', alpha=0.9, boxstyle='round,pad=0.5'))
        
        plt.savefig(f"{output_dir}/12_best_parameters.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # 11. Generate a summary HTML report
    try:
        # Create an HTML summary that links to all the images
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Clustering Results Summary</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}
                h1, h2 {{ color: #333366; }}
                .container {{ max-width: 1200px; margin: auto; background-color: white; padding: 20px; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
                .img-container {{ margin: 20px 0; text-align: center; }}
                img {{ max-width: 100%; border: 1px solid #ddd; border-radius: 5px; }}
                .summary {{ background-color: #e6f7ff; padding: 15px; border-radius: 5px; margin-bottom: 20px; }}
                table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
                th, td {{ padding: 10px; border: 1px solid #ddd; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                a {{ color: #0066cc; }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Clustering Analysis Results</h1>
                
                <div class="summary">
                    <h2>Summary</h2>
                    <p><strong>Total Samples:</strong> {total_samples}</p>
                    <p><strong>Number of Clusters:</strong> {n_clusters}</p>
                    <p><strong>Largest Cluster Size:</strong> {max(densities)}</p>
                    <p><strong>Average Cluster Size:</strong> {np.mean(densities):.2f}</p>
                    <p><strong>Median Cluster Size:</strong> {np.median(densities):.2f}</p>
                </div>
                
                <h2>Visualizations</h2>
        """
        
        # Add all the images
        images = [
            ("1_cluster_size_distribution.png", "Cluster Size Distribution"),
            ("2_top_5_clusters.png", "Top 5 Largest Clusters"),
            ("3_cumulative_distribution.png", "Cumulative Sample Distribution"),
            ("4_similarity_heatmap.png", "Similarity Between Top Clusters"),
            ("5_pca_visualization.png", "PCA Visualization"),
            ("6_tsne_visualization.png", "t-SNE Visualization"),
            ("7_cluster_quality_metrics.png", "Cluster Quality Metrics"),
            ("8_top_5_clusters_analysis.png", "Detailed Analysis of Top 5 Clusters")
        ]
        
        if all_results is not None:
            images.extend([
                ("9_parameter_heatmap.png", "Parameter Heatmap"),
                ("10_parameter_impact.png", "Parameter Impact on Cluster Count"),
                ("11_parameter_space.png", "Parameter Space Exploration"),
                ("12_best_parameters.png", "Best Parameter Combinations")
            ])
        
        for img_file, img_title in images:
            html_content += f"""
                <div class="img-container">
                    <h3>{img_title}</h3>
                    <a href="{img_file}" target="_blank">
                        <img src="{img_file}" alt="{img_title}">
                    </a>
                    <p><a href="{img_file}" download>Download this image</a></p>
                </div>
            """
        
        # Add top clusters table
        html_content += """
                <h2>Top 5 Clusters</h2>
                <table>
                    <tr>
                        <th>Cluster ID</th>
                        <th>Size</th>
                        <th>% of Total</th>
        """
        
        if similarity_matrix is not None:
            html_content += """
                        <th>Avg. Intra-Similarity</th>
            """
        
        html_content += """
                    </tr>
        """
        
        # Add data for top 5 clusters
        top_5_clusters = np.argsort(densities)[::-1][:5]
        for cluster_idx in top_5_clusters:
            cluster_size = densities[cluster_idx]
            percentage = (cluster_size / total_samples) * 100
            
            html_content += f"""
                    <tr>
                        <td>{cluster_idx}</td>
                        <td>{cluster_size}</td>
                        <td>{percentage:.2f}%</td>
            """
            
            if similarity_matrix is not None:
                # Calculate average intra-cluster similarity
                cluster = clusters[cluster_idx]
                if len(cluster) > 1:
                    intra_similarities = []
                    for idx_i in range(len(cluster)):
                        for idx_j in range(idx_i+1, len(cluster)):
                            intra_similarities.append(similarity_matrix[cluster[idx_i], cluster[idx_j]])
                    avg_sim = np.mean(intra_similarities) if intra_similarities else 0
                else:
                    avg_sim = 1.0  # Single-element cluster
                
                html_content += f"""
                        <td>{avg_sim:.4f}</td>
                """
            
            html_content += """
                    </tr>
            """
        
        html_content += """
                </table>
        """
        
        # Close tags
        html_content += """
            </div>
        </body>
        </html>
        """
        
        # Write the HTML file
        with open(f"{output_dir}/clustering_report.html", "w") as f:
            f.write(html_content)
        
        print(f"HTML report generated: {output_dir}/clustering_report.html")
    except Exception as e:
        print(f"Error generating HTML report: {e}")
    
    print(f"All visualizations saved to {output_dir}/")
    return True

# Example usage:
visualize_clustering_detailed(
    clusters=clusters, 
    densities=densities,
    features_df=features_df, 
    similarity_matrix=similarity,  # Pass the similarity matrix
    params=best_params,  # Pass the parameters used
    all_results=all_results,  # Pass parameter exploration results
    output_dir="cluster_analysis"  # Output directory
)

Created output directory: cluster_analysis
Visualizing clustering results for 11 clusters with 624 total samples


/var/folders/px/5k5grmpn7m1683gh7vt2xhsw0000gn/T/ipykernel_6036/2730747601.py:162: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "b-" (-> color='b'). The keyword argument will take precedence.
  ax.plot(
/var/folders/px/5k5grmpn7m1683gh7vt2xhsw0000gn/T/ipykernel_6036/2730747601.py:294: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = get_cmap('tab20')
/Users/ankitaniket/Downloads/Final Year Project/clustering-algo-code/.venv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


Error calculating external validation metrics: only integer scalar arrays can be converted to a scalar index
HTML report generated: cluster_analysis/clustering_report.html
All visualizations saved to cluster_analysis/


True

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

def visualize_clusters(clusters, labels, ari, nmi):
    cluster_purities = []
    cluster_sizes = []
    cluster_compositions = []

    for cluster in clusters:
        true_labels = labels[cluster]
        label_counts = Counter(true_labels)
        most_common_label, count = label_counts.most_common(1)[0]
        purity = count / len(cluster) * 100
        cluster_purities.append(purity)
        cluster_sizes.append(len(cluster))
        cluster_compositions.append(label_counts)

    # Purity Bar Plot
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(range(len(clusters))), y=cluster_purities)
    plt.ylabel('Purity (%)')
    plt.xlabel('Cluster Index')
    plt.title('Cluster Purity (Higher is Better)')
    plt.show()

    # Cluster Size Bar Plot
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(range(len(clusters))), y=cluster_sizes)
    plt.ylabel('Number of Samples')
    plt.xlabel('Cluster Index')
    plt.title('Cluster Sizes')
    plt.show()

    # Pie Chart of top 3 biggest clusters
    for i in np.argsort(cluster_sizes)[-3:][::-1]:
        labels_, counts = zip(*cluster_compositions[i].items())
        plt.figure()
        plt.pie(counts, labels=labels_, autopct='%1.1f%%', startangle=140)
        plt.title(f"Cluster {i} Label Distribution")
        plt.axis('equal')
        plt.show()

    # Print Easy-to-Understand Summary
    print("\n📊 Clustering Evaluation Summary:")
    print(f"- Adjusted Rand Index (ARI): {ari:.3f} → Low overlap with true labels.")
    print(f"- Normalized Mutual Information (NMI): {nmi:.3f} → Low mutual information.")
    print(f"- Average Cluster Purity: {np.mean(cluster_purities):.2f}%")


# Best Cluster result

In [ ]:
def visualize_best_cluster_analysis(clusters, densities, features_df, similarity_matrix=None, labels=None, output_dir="best_cluster_analysis"):
    """
    Creates specialized visualizations focusing on the best clusters from your algorithm.
    
    Args:
        clusters: List of clusters (each cluster is a list of sample indices)
        densities: List of cluster sizes
        features_df: DataFrame containing the original features
        similarity_matrix: Similarity matrix used for clustering (optional)
        labels: Optional ground truth labels for evaluation
        output_dir: Directory to save visualizations (will be created if it doesn't exist)
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import seaborn as sns
    from sklearn.manifold import TSNE
    from sklearn.decomposition import PCA
    from sklearn.metrics import silhouette_samples, silhouette_score
    from sklearn.preprocessing import StandardScaler
    import matplotlib.gridspec as gridspec
    from matplotlib.colors import ListedColormap
    import matplotlib.patches as mpatches
    import os
    from matplotlib.cm import get_cmap
    from scipy.cluster.hierarchy import linkage, dendrogram
    import matplotlib.ticker as ticker
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    
    # Set overall styling for plots
    plt.style.use('ggplot')
    sns.set_style("whitegrid")
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans', 'Liberation Sans', 'Bitstream Vera Sans', 'sans-serif']
    
    # Basic statistics
    n_clusters = len(clusters)
    total_samples = sum(densities)
    print(f"Analyzing best clusters out of {n_clusters} total clusters with {total_samples} total samples")
    
    # Assign cluster labels to each sample
    sample_to_cluster = np.zeros(features_df.shape[0], dtype=int) - 1  # Initialize with -1 (no cluster)
    for i, cluster in enumerate(clusters):
        for sample_idx in cluster:
            sample_to_cluster[sample_idx] = i
    
    # Identify top clusters based on size and quality
    top_clusters_idx = np.argsort(densities)[::-1][:10]  # Top 10 by size
    
    # If similarity matrix is available, also calculate cohesion scores
    if similarity_matrix is not None:
        cohesion_scores = []
        for i, cluster in enumerate(clusters):
            if len(cluster) > 1:
                # Average similarity within cluster
                cluster_sim = 0
                count = 0
                for idx_i in range(len(cluster)):
                    for idx_j in range(idx_i+1, len(cluster)):
                        cluster_sim += similarity_matrix[cluster[idx_i], cluster[idx_j]]
                        count += 1
                        
                cohesion = cluster_sim / count if count > 0 else 0
            else:
                cohesion = 1.0  # Single-element cluster
                
            cohesion_scores.append(cohesion)
        
        # Identify top cohesive clusters (may overlap with largest)
        top_cohesive_idx = np.argsort(cohesion_scores)[::-1][:10]
        
        # Calculate combined score (size * cohesion) to find "best" clusters
        combined_scores = np.array(densities) * np.array(cohesion_scores)
        combined_top_idx = np.argsort(combined_scores)[::-1][:10]
        
        # Make a union of all these top clusters
        best_clusters_idx = np.unique(np.concatenate([top_clusters_idx[:5], top_cohesive_idx[:5], combined_top_idx[:5]]))
    else:
        # Without similarity matrix, just use size
        best_clusters_idx = top_clusters_idx[:10]
    
    # =============== 1. Overview of Best Clusters ===============
    # Create a summary visualization of the best clusters
    plt.figure(figsize=(14, 10))
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((2, 2), (1, 0))
    ax3 = plt.subplot2grid((2, 2), (1, 1))
    
    # Bar chart of best cluster sizes
    colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(best_clusters_idx)))
    bars = ax1.bar(best_clusters_idx, [densities[i] for i in best_clusters_idx], color=colors)
    
    # Add values on top of bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax1.text(
            bar.get_x() + bar.get_width()/2.,
            height + 5,
            f'{int(height)}',
            ha='center', 
            va='bottom',
            fontsize=12,
            fontweight='bold'
        )
    
    # Add percentage labels inside bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        percentage = (height / total_samples) * 100
        ax1.text(
            bar.get_x() + bar.get_width()/2.,
            height / 2,
            f'{percentage:.1f}%',
            ha='center', 
            va='center',
            fontsize=11,
            fontweight='bold',
            color='white'
        )
    
    ax1.set_title("Best Clusters by Size", fontsize=16)
    ax1.set_xlabel("Cluster ID", fontsize=14)
    ax1.set_ylabel("Number of Samples", fontsize=14)
    
    # Calculate total percentage covered by best clusters
    best_total = sum(densities[i] for i in best_clusters_idx)
    best_percent = (best_total / total_samples) * 100
    ax1.text(
        0.5, 0.95,
        f"These {len(best_clusters_idx)} clusters contain {best_percent:.1f}% of all samples",
        transform=ax1.transAxes,
        ha='center',
        fontsize=13,
        fontweight='bold',
        bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.5')
    )
    
    # Pie chart showing proportion of samples in best clusters vs other clusters
    sizes = [best_total, total_samples - best_total]
    labels = [f'Best Clusters ({best_percent:.1f}%)', f'Other Clusters ({100-best_percent:.1f}%)']
    colors = ['#3498db', '#e74c3c']
    
    ax2.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', 
           startangle=90, shadow=True, explode=(0.1, 0))
    ax2.set_title("Distribution of Samples", fontsize=14)
    
    # Scatter plot of cluster size vs cohesion (if available)
    if similarity_matrix is not None:
        ax3.scatter(densities, cohesion_scores, s=100, alpha=0.7, c=range(len(densities)), cmap='viridis')
        
        # Highlight best clusters
        for idx in best_clusters_idx:
            ax3.scatter(densities[idx], cohesion_scores[idx], s=200, edgecolor='red', 
                      facecolor='none', linewidth=2, marker='o')
            ax3.text(densities[idx]+5, cohesion_scores[idx], f'{idx}', fontsize=12)
        
        ax3.set_title("Cluster Size vs. Cohesion", fontsize=14)
        ax3.set_xlabel("Cluster Size", fontsize=12)
        ax3.set_ylabel("Cohesion Score", fontsize=12)
        
        # Add quadrant labels for interpretation
        ax3.axhline(np.median(cohesion_scores), color='gray', linestyle='--', alpha=0.7)
        ax3.axvline(np.median(densities), color='gray', linestyle='--', alpha=0.7)
        
        # Add quadrant annotations
        quadrants = [
            (0.25, 0.25, "Small\nLow Cohesion"),
            (0.25, 0.75, "Small\nHigh Cohesion"),
            (0.75, 0.25, "Large\nLow Cohesion"),
            (0.75, 0.75, "Large\nHigh Cohesion")
        ]
        
        for x, y, text in quadrants:
            ax3.text(
                x, y, text,
                transform=ax3.transAxes,
                ha='center', va='center',
                fontsize=10, alpha=0.7,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.6)
            )
    else:
        ax3.text(0.5, 0.5, "Similarity matrix not provided", 
                ha='center', va='center', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/1_best_clusters_overview.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # =============== 2. Dimensionality Reduction: Best Clusters Only ===============
    # Convert features to numpy for dimensionality reduction
    X = features_df.values
    
    # Apply PCA to visualize best clusters
    plt.figure(figsize=(16, 8))
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2)
    
    # Apply PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    # Get a colormap with distinct colors
    colors = plt.cm.tab20(np.linspace(0, 1, len(best_clusters_idx)))
    
    # Create list for legend
    legend_elements = []
    
    # Plot unclustered or non-best cluster points first
    mask = np.ones(len(sample_to_cluster), dtype=bool)
    for idx in best_clusters_idx:
        cluster_samples = np.where(sample_to_cluster == idx)[0]
        mask[cluster_samples] = False
    
    if np.any(mask):
        ax1.scatter(
            X_pca[mask, 0], 
            X_pca[mask, 1], 
            c='lightgray', 
            alpha=0.3, 
            s=20,
            label="Other clusters"
        )
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor='lightgray', 
                                     label=f'Other clusters', 
                                     markersize=8))
    
    # Plot each best cluster with distinct color
    for i, cluster_idx in enumerate(best_clusters_idx):
        cluster_samples = np.where(sample_to_cluster == cluster_idx)[0]
        ax1.scatter(
            X_pca[cluster_samples, 0], 
            X_pca[cluster_samples, 1], 
            c=[colors[i]], 
            alpha=0.7, 
            s=50,
            label=f"Cluster {cluster_idx} (size: {densities[cluster_idx]})"
        )
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                     markerfacecolor=colors[i], 
                                     label=f'Cluster {cluster_idx} (size: {densities[cluster_idx]})', 
                                     markersize=8))
    
    # Add variance explained
    var_explained = pca.explained_variance_ratio_
    ax1.set_xlabel(f"PC1 ({var_explained[0]:.2%} variance)", fontsize=14)
    ax1.set_ylabel(f"PC2 ({var_explained[1]:.2%} variance)", fontsize=14)
    ax1.set_title("PCA: Best Clusters", fontsize=16)
    
    # Add legend with better positioning
    ax1.legend(
        handles=legend_elements, 
        title="Clusters", 
        loc="upper right",
        bbox_to_anchor=(1.05, 1),
        fontsize=10
    )
    
    # t-SNE visualization of best clusters
    try:
        # Limit samples for t-SNE if too many
        max_tsne_samples = 5000
        if X.shape[0] > max_tsne_samples:
            np.random.seed(42)
            sample_indices = np.random.choice(X.shape[0], max_tsne_samples, replace=False)
            X_subset = X[sample_indices]
            clusters_subset = sample_to_cluster[sample_indices]
        else:
            X_subset = X
            clusters_subset = sample_to_cluster
        
        # Apply t-SNE with good parameters
        tsne = TSNE(
            n_components=2, 
            perplexity=min(40, len(X_subset)-1), 
            n_iter=1000,
            random_state=42
        )
        X_tsne = tsne.fit_transform(X_subset)
        
        # Create mask for best clusters in the t-SNE subset
        mask_tsne = np.ones(len(clusters_subset), dtype=bool)
        legend_elements_tsne = []
        
        # Plot background points
        for idx in best_clusters_idx:
            cluster_samples = np.where(clusters_subset == idx)[0]
            mask_tsne[cluster_samples] = False
        
        if np.any(mask_tsne):
            ax2.scatter(
                X_tsne[mask_tsne, 0], 
                X_tsne[mask_tsne, 1], 
                c='lightgray', 
                alpha=0.3, 
                s=20,
                label="Other clusters"
            )
            legend_elements_tsne.append(plt.Line2D([0], [0], marker='o', color='w', 
                                         markerfacecolor='lightgray', 
                                         label=f'Other clusters', 
                                         markersize=8))
        
        # Plot best clusters in t-SNE space
        for i, cluster_idx in enumerate(best_clusters_idx):
            cluster_samples = np.where(clusters_subset == cluster_idx)[0]
            if len(cluster_samples) > 0:
                ax2.scatter(
                    X_tsne[cluster_samples, 0], 
                    X_tsne[cluster_samples, 1], 
                    c=[colors[i]], 
                    alpha=0.7, 
                    s=50,
                    label=f"Cluster {cluster_idx}"
                )
                legend_elements_tsne.append(plt.Line2D([0], [0], marker='o', color='w', 
                                             markerfacecolor=colors[i], 
                                             label=f'Cluster {cluster_idx}', 
                                             markersize=8))
        
        ax2.set_title("t-SNE: Best Clusters", fontsize=16)
        
        # Add legend
        ax2.legend(
            handles=legend_elements_tsne, 
            title="Clusters", 
            loc="upper right",
            bbox_to_anchor=(1.05, 1),
            fontsize=10
        )
    except Exception as e:
        ax2.text(0.5, 0.5, f"t-SNE error: {str(e)}", 
                ha='center', va='center', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/2_best_clusters_visualization.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # =============== 3. Feature Analysis for Best Clusters ===============
    if features_df.shape[1] > 0:  # Make sure we have features
        # Calculate which features best discriminate the top clusters
        # Convert to DataFrame for easier analysis
        X_df = features_df.copy()
        X_df['cluster'] = sample_to_cluster
        
        # Filter to best clusters only
        best_clusters_mask = np.isin(X_df['cluster'], best_clusters_idx)
        X_best = X_df[best_clusters_mask]
        
        # For each feature, calculate difference between clusters
        feature_importance = {}
        importance_data = []
        
        # Limit to 20 features for visualization
        feature_cols = features_df.columns[:20] if len(features_df.columns) > 20 else features_df.columns
        
        for col in feature_cols:
            # Skip if not numeric
            if not np.issubdtype(X_df[col].dtype, np.number):
                continue
                
            # Calculate mean value for each cluster
            cluster_means = {}
            for cluster_idx in best_clusters_idx:
                cluster_data = X_best[X_best['cluster'] == cluster_idx][col]
                if len(cluster_data) > 0:
                    cluster_means[cluster_idx] = cluster_data.mean()
            
            # Skip if any cluster has no data
            if len(cluster_means) < len(best_clusters_idx):
                continue
                
            # Calculate variance ratio: between-cluster variance / within-cluster variance
            global_mean = X_best[col].mean()
            
            # Between-cluster variance
            between_var = sum((cluster_means[idx] - global_mean) ** 2 for idx in cluster_means) / len(cluster_means)
            
            # Within-cluster variance
            within_var_total = 0
            within_var_count = 0
            
            for cluster_idx in best_clusters_idx:
                cluster_data = X_best[X_best['cluster'] == cluster_idx][col]
                if len(cluster_data) > 1:  # Need at least 2 points for variance
                    within_var_total += np.var(cluster_data) * (len(cluster_data) - 1)
                    within_var_count += len(cluster_data) - 1
            
            # Avoid division by zero
            if within_var_count > 0 and within_var_total > 0:
                within_var = within_var_total / within_var_count
                f_ratio = between_var / within_var
            else:
                f_ratio = 0
                
            feature_importance[col] = f_ratio
            
            # Store data for visualization
            for cluster_idx in best_clusters_idx:
                cluster_data = X_best[X_best['cluster'] == cluster_idx][col]
                if len(cluster_data) > 0:
                    importance_data.append({
                        'Feature': col,
                        'Cluster': f'Cluster {cluster_idx}',
                        'Value': cluster_data.mean(),
                        'F-ratio': f_ratio
                    })
        
        # Convert to DataFrame for easier plotting
        importance_df = pd.DataFrame(importance_data)
        
        # Get top 10 most discriminative features
        top_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:10]
        top_feature_names = [f[0] for f in top_features]
        
        # Plot feature importance
        plt.figure(figsize=(16, 10))
        ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
        ax2 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
        
        # Feature importance bar chart
        feature_imp_values = [f[1] for f in top_features]
        feature_names = [str(f[0])[:20] for f in top_features]  # Truncate long names
        
        bars = ax1.bar(feature_names, feature_imp_values, color='royalblue')
        
        # Add values on top of bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax1.text(
                bar.get_x() + bar.get_width()/2.,
                height + 0.1,
                f'{height:.2f}',
                ha='center', 
                va='bottom',
                fontsize=10
            )
        
        ax1.set_title("Top 10 Most Discriminative Features", fontsize=16)
        ax1.set_xlabel("Feature", fontsize=14)
        ax1.set_ylabel("F-ratio (Between/Within Variance)", fontsize=14)
        plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
        
        # Feature values across clusters heatmap
        if importance_df.empty or len(top_feature_names) == 0:
            ax2.text(0.5, 0.5, "Not enough data for feature analysis", 
                    ha='center', va='center', fontsize=14)
        else:
            # Filter to top features
            plot_df = importance_df[importance_df['Feature'].isin(top_feature_names)]
            
            if not plot_df.empty:
                # Create pivot table
                pivot = plot_df.pivot_table(
                    index='Feature', 
                    columns='Cluster', 
                    values='Value',
                    aggfunc='mean'
                )
                
                # Normalize for better visualization
                pivot_norm = (pivot - pivot.min()) / (pivot.max() - pivot.min())
                
                # Plot heatmap
                sns.heatmap(
                    pivot_norm, 
                    annot=True, 
                    cmap="YlGnBu", 
                    ax=ax2, 
                    fmt='.2f', 
                    cbar_kws={'label': 'Normalized Value'}
                )
                
                ax2.set_title("Feature Values Across Best Clusters (Normalized)", fontsize=16)
                ax2.set_ylabel("Feature", fontsize=14)
                ax2.set_xlabel("Cluster", fontsize=14)
            else:
                ax2.text(0.5, 0.5, "Not enough data for feature analysis", 
                        ha='center', va='center', fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/3_feature_analysis.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        # =============== 4. Feature Distribution Within Best Clusters ===============
        # Create violin plots for top 5 most important features across clusters
        if len(top_feature_names) > 0:
            top_5_features = top_feature_names[:5]
            
            plt.figure(figsize=(16, 12))
            
            for i, feature in enumerate(top_5_features):
                ax = plt.subplot(3, 2, i+1)
                
                # Data for plot
                plot_data = []
                plot_labels = []
                
                for cluster_idx in best_clusters_idx:
                    cluster_data = X_df[X_df['cluster'] == cluster_idx][feature].values
                    if len(cluster_data) > 0:
                        plot_data.append(cluster_data)
                        plot_labels.append(f'Cluster {cluster_idx}')
                
                if len(plot_data) > 0:
                    # Create violin plot
                    parts = ax.violinplot(
                        plot_data, 
                        showmeans=True, 
                        showmedians=True,
                        showextrema=True
                    )
                    
                    # Customize violin plots
                    for pc, color in zip(parts['bodies'], colors[:len(plot_data)]):
                        pc.set_facecolor(color)
                        pc.set_alpha(0.7)
                    
                    # Set labels and title
                    ax.set_xticks(range(1, len(plot_labels) + 1))
                    ax.set_xticklabels(plot_labels, rotation=45, ha='right')
                    ax.set_title(f"Distribution of {feature}", fontsize=14)
                    ax.set_ylabel("Feature Value", fontsize=12)
                else:
                    ax.text(0.5, 0.5, "Not enough data for feature analysis", 
                           ha='center', va='center', fontsize=14)
            
            plt.tight_layout()
            plt.savefig(f"{output_dir}/4_feature_distributions.png", dpi=300, bbox_inches='tight')
            plt.close()
    
    # =============== 5. Cluster Similarity Analysis ===============
    if similarity_matrix is not None:
        plt.figure(figsize=(14, 12))
        ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
        ax2 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
        
        # Calculate similarity between best clusters
        best_cluster_similarity = np.zeros((len(best_clusters_idx), len(best_clusters_idx)))
        
        for i, cluster_i_idx in enumerate(best_clusters_idx):
            for j, cluster_j_idx in enumerate(best_clusters_idx):
                cluster_i = clusters[cluster_i_idx]
                cluster_j = clusters[cluster_j_idx]
                
                # Get all pairwise similarities between samples in the two clusters
                similarities = []
                for idx_i in cluster_i:
                    for idx_j in cluster_j:
                        similarities.append(similarity_matrix[idx_i, idx_j])
                
                # Calculate average similarity
                best_cluster_similarity[i, j] = np.mean(similarities) if similarities else 0
        
        # Create custom labels showing cluster index and size
        labels = [f"{idx}\n(size: {densities[idx]})" for idx in best_clusters_idx]
        
        # Plot heatmap
        sns.heatmap(
            best_cluster_similarity, 
            annot=True, 
            cmap="YlGnBu", 
            ax=ax1, 
            vmin=0, 
            vmax=1, 
            fmt='.2f', 
            xticklabels=labels, 
            yticklabels=labels,
            linewidths=0.5,
            annot_kws={"size": 10, "weight": "bold"}
        )
        
        ax1.set_title("Similarity Between Best Clusters", fontsize=16)
        ax1.set_xlabel("Cluster ID (size)", fontsize=14)
        ax1.set_ylabel("Cluster ID (size)", fontsize=14)
        
        # Create hierarchical clustering of the clusters based on similarity
        # Convert similarity to distance (1 - similarity)
        distance_matrix = 1 - best_cluster_similarity
        
        # Compute linkage
        Z = linkage(distance_matrix, method='ward')
        
        # Plot dendrogram
        dendrogram(
            Z, 
            labels=[f"Cluster {idx}" for idx in best_clusters_idx],
            ax=ax2, 
            orientation='top',
            leaf_font_size=12,
            color_threshold=0.5  # adjust this threshold as needed
        )
        
        ax2.set_title("Hierarchical Clustering of Best Clusters", fontsize=16)
        ax2.set_ylabel("Distance (1 - Similarity)", fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/5_cluster_similarity.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # =============== 6. Silhouette Analysis of Best Clusters ===============
    if similarity_matrix is not None:
        plt.figure(figsize=(14, 10))
        
        # Create a filter for samples in best clusters
        best_clusters_mask = np.isin(sample_to_cluster, best_clusters_idx)
        best_cluster_samples = np.where(best_clusters_mask)[0]
        
        if len(best_cluster_samples) > 1:
            # Convert similarity to distance for silhouette
            distance_matrix = 1 - similarity_matrix
            
            # Filter matrices for best clusters only
            best_distances = distance_matrix[best_cluster_samples][:, best_cluster_samples]
            best_labels = sample_to_cluster[best_cluster_samples]
            
            # Compute silhouette scores
            try:
                silhouette_vals = silhouette_samples(
                    best_distances, 
                    best_labels, 
                    metric='precomputed'
                )
                
                # Compute average silhouette score
                avg_silhouette = silhouette_score(
                    best_distances, 
                    best_labels, 
                    metric='precomputed'
                )
                
                # Plot silhouette
                y_lower = 10
                ax = plt.gca()
                
                # Sort clusters by size (largest first)
                cluster_sizes = {}
                for idx in best_clusters_idx:
                    cluster_sizes[idx] = np.sum(best_labels == idx)
                    
                sorted_clusters = sorted(cluster_sizes.items(), key=lambda x: x[1], reverse=True)
                sorted_idx = [item[0] for item in sorted_clusters]
                
                # Plot each cluster's silhouette
                for i, cluster_idx in enumerate(sorted_idx):
                    # Get silhouette values for this cluster
                    cluster_silhouette_vals = silhouette_vals[best_labels == cluster_idx]
                    cluster_silhouette_vals.sort()
                    
                    size = cluster_silhouette_vals.shape[0]
                    if size == 0:
                        continue
                        
                    y_upper = y_lower + size
                    
                    color = plt.cm.nipy_spectral(i / len(sorted_idx))
                    ax.fill_betweenx(
                        np.arange(y_lower, y_upper),
                        0, 
                        cluster_silhouette_vals,
                        facecolor=color, 
                        edgecolor=color, 
                        alpha=0.7
                    )
                    
                    # Label cluster
                    ax.text(
                        -0.05, 
                        y_lower + 0.5 * size, 
                        f'Cluster {cluster_idx}\n({size} samples)',
                        fontsize=10, 
                        verticalalignment='center'
                    )
                    
                    # Compute next y_lower
                    y_lower = y_upper + 10
                
                # Add average silhouette line
                ax.axvline(
                    x=avg_silhouette, 
                    color="red", 
                    linestyle="--",
                    label=f'Average: {avg_silhouette:.3f}'
                )
                
                ax.set_title("Silhouette Analysis of Best Clusters", fontsize=16)
                ax.set_xlabel("Silhouette Coefficient", fontsize=14)
                ax.set_ylabel("Cluster", fontsize=14)
                
                # Set limits
                ax.set_xlim([-0.1, 1])
                
                # Add legend
                ax.legend(loc='best', fontsize=12)
                # Add interpretive labels
                silhouette_ranges = [
                    (-0.1, 0.25, "Poor separation", "lightcoral"),
                    (0.25, 0.5, "Fair separation", "khaki"),
                    (0.5, 0.75, "Good separation", "palegreen"),
                    (0.75, 1.0, "Excellent separation", "lightblue")
                ]
                
                for start, end, label, color in silhouette_ranges:
                    ax.axvspan(start, end, alpha=0.2, color=color)
                    ax.text(
                        (start + end) / 2, 
                        -0.05, 
                        label, 
                        ha='center', 
                        va='top', 
                        fontsize=10,
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
                    )
                
            except Exception as e:
                plt.clf()  # Clear the figure
                ax = plt.gca()
                ax.text(0.5, 0.5, f"Error computing silhouette: {str(e)}", 
                       ha='center', va='center', fontsize=14)
        else:
            ax = plt.gca()
            ax.text(0.5, 0.5, "Not enough samples for silhouette analysis", 
                   ha='center', va='center', fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/6_silhouette_analysis.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # =============== 7. Cluster Stability Analysis ===============
    # This simulates how stable your clusters are by sampling
    plt.figure(figsize=(14, 10))
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((2, 2), (1, 0), colspan=1)
    ax3 = plt.subplot2grid((2, 2), (1, 1), colspan=1)
    
    # Simulate stability by subsampling
    n_iterations = 20
    stability_scores = {idx: [] for idx in best_clusters_idx}
    overlap_scores = []
    
    try:
        # Subsampling fraction (e.g., 80% of data)
        subsample_fraction = 0.8
        
        for _ in range(n_iterations):
            # Subsample the data
            n_samples = features_df.shape[0]
            subsample_size = int(n_samples * subsample_fraction)
            
            # Create random subsample
            np.random.seed(_ + 42)  # Different seed each time
            subsample_indices = np.random.choice(n_samples, subsample_size, replace=False)
            
            # Calculate overlap of subsample with each best cluster
            for cluster_idx in best_clusters_idx:
                cluster = clusters[cluster_idx]
                
                # Calculate how many samples from the cluster are in the subsample
                overlap = np.intersect1d(cluster, subsample_indices)
                
                # Calculate proportion of cluster preserved in subsample
                if len(cluster) > 0:
                    stability = len(overlap) / len(cluster)
                else:
                    stability = 0
                    
                stability_scores[cluster_idx].append(stability)
            
            # Calculate overall average overlap across all best clusters
            overlap_scores.append(np.mean([stability_scores[idx][-1] for idx in best_clusters_idx]))
        
        # Plot stability scores
        stability_means = [np.mean(stability_scores[idx]) for idx in best_clusters_idx]
        stability_stds = [np.std(stability_scores[idx]) for idx in best_clusters_idx]
        
        # Bar chart of stability
        bars = ax1.bar(
            [f"Cluster {idx}" for idx in best_clusters_idx], 
            stability_means, 
            yerr=stability_stds,
            capsize=5,
            color=colors[:len(best_clusters_idx)]
        )
        
        # Add values on top of bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax1.text(
                bar.get_x() + bar.get_width()/2.,
                height + 0.02,
                f'{height:.2f}',
                ha='center', 
                va='bottom',
                fontsize=10
            )
        
        ax1.set_title("Cluster Stability Analysis", fontsize=16)
        ax1.set_xlabel("Cluster", fontsize=14)
        ax1.set_ylabel("Stability Score (Higher is Better)", fontsize=14)
        ax1.set_ylim(0, 1.1)
        
        # Add interpretive thresholds
        ax1.axhline(0.9, color='green', linestyle='--', alpha=0.7, label='Excellent')
        ax1.axhline(0.7, color='orange', linestyle='--', alpha=0.7, label='Good')
        ax1.axhline(0.5, color='red', linestyle='--', alpha=0.7, label='Fair')
        ax1.legend(fontsize=12)
        
        # Plot histogram of stability scores for the two most stable clusters
        sorted_stability = sorted([(idx, np.mean(scores)) for idx, scores in stability_scores.items()], 
                                 key=lambda x: x[1], reverse=True)
        
        if len(sorted_stability) >= 1:
            most_stable_idx = sorted_stability[0][0]
            
            ax2.hist(
                stability_scores[most_stable_idx], 
                bins=10, 
                alpha=0.7, 
                color=colors[0],
                edgecolor='black'
            )
            
            ax2.set_title(f"Stability Distribution: Cluster {most_stable_idx}", fontsize=14)
            ax2.set_xlabel("Stability Score", fontsize=12)
            ax2.set_ylabel("Frequency", fontsize=12)
            ax2.set_xlim(0, 1)
        
        # Plot trend of overall overlap across iterations
        ax3.plot(
            range(1, n_iterations + 1),
            overlap_scores,
            marker='o',
            linestyle='-',
            color='royalblue',
            linewidth=2
        )
        
        ax3.set_title("Overall Cluster Stability Trend", fontsize=14)
        ax3.set_xlabel("Iteration", fontsize=12)
        ax3.set_ylabel("Average Stability", fontsize=12)
        ax3.set_ylim(0, 1)
        ax3.grid(True, linestyle='--', alpha=0.7)
        
    except Exception as e:
        for ax in [ax1, ax2, ax3]:
            ax.text(0.5, 0.5, f"Error in stability analysis: {str(e)}", 
                   ha='center', va='center', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/7_stability_analysis.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # =============== 8. Top 5 Best Clusters Detail Cards ===============
    # Create detailed "profile cards" for the top 5 best clusters
    
    # Get top 5 clusters by combined score (size * cohesion)
    if similarity_matrix is not None and len(cohesion_scores) > 0:
        combined_scores = np.array(densities) * np.array(cohesion_scores)
        top5_idx = np.argsort(combined_scores)[::-1][:5]
    else:
        # Without similarity, just use size
        top5_idx = np.argsort(densities)[::-1][:5]
    
    # Create a detail card for each top cluster
    plt.figure(figsize=(20, 25))
    
    # Use gridspec for more control
    gs = gridspec.GridSpec(5, 1, height_ratios=[1, 1, 1, 1, 1], hspace=0.4)
    
    for i, cluster_idx in enumerate(top5_idx):
        # Create a subplot for this cluster
        ax_main = plt.subplot(gs[i])
        
        # Calculate cluster stats
        cluster = clusters[cluster_idx]
        cluster_size = len(cluster)
        percentage = (cluster_size / total_samples) * 100
        
        # Get cohesion score if available
        if similarity_matrix is not None and i < len(cohesion_scores):
            cohesion = cohesion_scores[cluster_idx]
        else:
            cohesion = None
        
        # Create a profile card layout using gridspec
        # This creates a card with stats and visualizations
        ax_main.axis('off')
        
        # Use a light background for the card
        ax_main.add_patch(plt.Rectangle(
            (0, 0), 1, 1, 
            transform=ax_main.transAxes,
            facecolor='#f5f5f5', 
            edgecolor='#cccccc',
            linewidth=2,
            zorder=-1
        ))
        
        # Add cluster title
        ax_main.text(
            0.5, 0.95,
            f"Cluster {cluster_idx} Profile",
            transform=ax_main.transAxes,
            ha='center',
            va='top',
            fontsize=18,
            fontweight='bold',
            bbox=dict(facecolor='#3498db', alpha=0.7, boxstyle='round,pad=0.3', edgecolor='none'),
            color='white'
        )
        
        # Add basic stats
        stats_text = (
            f"Size: {cluster_size} samples ({percentage:.1f}% of total)\n"
        )
        
        if cohesion is not None:
            stats_text += f"Cohesion: {cohesion:.3f}\n"
            
            # Add qualitative assessment of cohesion
            if cohesion > 0.8:
                cohesion_quality = "Excellent (highly similar samples)"
            elif cohesion > 0.6:
                cohesion_quality = "Good (mostly similar samples)"
            elif cohesion > 0.4:
                cohesion_quality = "Moderate (somewhat similar samples)"
            else:
                cohesion_quality = "Low (diverse samples)"
                
            stats_text += f"Cohesion Quality: {cohesion_quality}\n"
        
        # Add stability if calculated
        if cluster_idx in stability_scores and stability_scores[cluster_idx]:
            stability = np.mean(stability_scores[cluster_idx])
            stats_text += f"Stability: {stability:.3f}\n"
            
            # Add qualitative assessment of stability
            if stability > 0.9:
                stability_quality = "Excellent (very stable cluster)"
            elif stability > 0.7:
                stability_quality = "Good (stable cluster)"
            elif stability > 0.5:
                stability_quality = "Moderate (somewhat stable cluster)"
            else:
                stability_quality = "Low (unstable cluster)"
                
            stats_text += f"Stability Quality: {stability_quality}\n"
        
        # Add feature importance if calculated
        if 'importance_df' in locals() and not importance_df.empty:
            # Get top 3 features for this cluster
            cluster_features = importance_df[importance_df['Cluster'] == f'Cluster {cluster_idx}']
            if not cluster_features.empty:
                top_cluster_features = cluster_features.sort_values('F-ratio', ascending=False).head(3)
                
                if not top_cluster_features.empty:
                    stats_text += "\nKey Features:\n"
                    for _, row in top_cluster_features.iterrows():
                        stats_text += f"- {row['Feature']}: {row['Value']:.3f}\n"
        
        # Add the stats text
        ax_main.text(
            0.05, 0.7,
            stats_text,
            transform=ax_main.transAxes,
            ha='left',
            va='top',
            fontsize=14,
            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3', edgecolor='#cccccc')
        )
        
        # Add a mini PCA plot for this cluster
        if X.shape[0] > 0 and X.shape[1] > 1:
            # Create a mini-axis for PCA
            ax_pca = plt.axes([0.6, 0.25 + i*0.2, 0.3, 0.15])
            
            # Get cluster samples
            cluster_samples = np.where(sample_to_cluster == cluster_idx)[0]
            
            # Plot all points in gray
            ax_pca.scatter(X_pca[:, 0], X_pca[:, 1], c='lightgray', alpha=0.3, s=10)
            
            # Highlight this cluster
            ax_pca.scatter(
                X_pca[cluster_samples, 0], 
                X_pca[cluster_samples, 1], 
                c=colors[i % len(colors)],
                alpha=0.7, 
                s=30,
                edgecolor='black'
            )
            
            ax_pca.set_title(f"PCA Visualization", fontsize=12)
            ax_pca.set_xticks([])
            ax_pca.set_yticks([])
        
        # Add a circular indicator for quality
        if cohesion is not None:
            quality_score = (cohesion + (stability if 'stability' in locals() else 0.5)) / 2
            
            # Create quality indicator
            ax_quality = plt.axes([0.45, 0.25 + i*0.2, 0.1, 0.15], polar=True)
            
            # Create a gauge chart
            theta = np.linspace(0, 2*np.pi, 100)
            r = np.ones_like(theta)
            
            # Background circle (gray)
            ax_quality.fill_between(theta, 0, r, color='lightgray', alpha=0.3)
            
            # Foreground circle (colored by quality)
            quality_theta = np.linspace(0, 2*np.pi*quality_score, 100)
            quality_r = np.ones_like(quality_theta)
            
            # Choose color based on quality
            if quality_score > 0.8:
                quality_color = 'green'
                quality_label = 'Excellent'
            elif quality_score > 0.6:
                quality_color = 'yellowgreen'
                quality_label = 'Good'
            elif quality_score > 0.4:
                quality_color = 'orange'
                quality_label = 'Moderate'
            else:
                quality_color = 'red'
                quality_label = 'Poor'
                
            ax_quality.fill_between(quality_theta, 0, quality_r, color=quality_color, alpha=0.7)
            
            # Remove ticks and spines
            ax_quality.set_xticks([])
            ax_quality.set_yticks([])
            ax_quality.spines['polar'].set_visible(False)
            
            # Add quality label
            ax_quality.text(
                0, 0,
                f"{quality_score:.2f}\n{quality_label}",
                ha='center',
                va='center',
                fontsize=10,
                fontweight='bold'
            )
            
            ax_quality.set_title("Quality", fontsize=12)
        
    plt.suptitle("Top 5 Best Clusters - Detailed Profiles", fontsize=20, y=0.98)
    plt.savefig(f"{output_dir}/8_top5_cluster_profiles.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # =============== 9. Generate a summary HTML report ===============
    try:
        # Create an HTML summary that links to all the images
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Best Clusters Analysis</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}
                h1, h2 {{ color: #333366; }}
                .container {{ max-width: 1200px; margin: auto; background-color: white; padding: 20px; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1); }}
                .img-container {{ margin: 20px 0; text-align: center; }}
                img {{ max-width: 100%; border: 1px solid #ddd; border-radius: 5px; }}
                .summary {{ background-color: #e6f7ff; padding: 15px; border-radius: 5px; margin-bottom: 20px; }}
                table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
                th, td {{ padding: 10px; border: 1px solid #ddd; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                a {{ color: #0066cc; }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Best Clusters Analysis</h1>
                
                <div class="summary">
                    <h2>Summary</h2>
                    <p><strong>Total Samples:</strong> {total_samples}</p>
                    <p><strong>Number of Clusters:</strong> {n_clusters}</p>
                    <p><strong>Number of Best Clusters Analyzed:</strong> {len(best_clusters_idx)}</p>
                    <p><strong>Percentage of Samples in Best Clusters:</strong> {best_percent:.2f}%</p>
                </div>
                
                <h2>Visualizations</h2>
        """
        
        # Add all the images
        images = [
            ("1_best_clusters_overview.png", "Best Clusters Overview"),
            ("2_best_clusters_visualization.png", "Best Clusters Visualization"),
            ("3_feature_analysis.png", "Feature Analysis"),
            ("4_feature_distributions.png", "Feature Distributions"),
            ("5_cluster_similarity.png", "Cluster Similarity Analysis"),
            ("6_silhouette_analysis.png", "Silhouette Analysis"),
            ("7_stability_analysis.png", "Stability Analysis"),
            ("8_top5_cluster_profiles.png", "Top 5 Cluster Profiles")
        ]
        
        for img_file, img_title in images:
            html_content += f"""
                <div class="img-container">
                    <h3>{img_title}</h3>
                    <a href="{img_file}" target="_blank">
                        <img src="{img_file}" alt="{img_title}">
                    </a>
                    <p><a href="{img_file}" download>Download this image</a></p>
                </div>
            """
        
        # Add top clusters table
        html_content += """
                <h2>Best Clusters Summary</h2>
                <table>
                    <tr>
                        <th>Cluster ID</th>
                        <th>Size</th>
                        <th>% of Total</th>
        """
        
        if similarity_matrix is not None:
            html_content += """
                        <th>Cohesion</th>
            """
            
        if 'stability_scores' in locals():
            html_content += """
                        <th>Stability</th>
            """
            
        html_content += """
                    </tr>
        """
        
        # Add data for best clusters
        for cluster_idx in best_clusters_idx:
            cluster_size = densities[cluster_idx]
            percentage = (cluster_size / total_samples) * 100
            
            html_content += f"""
                    <tr>
                        <td>{cluster_idx}</td>
                        <td>{cluster_size}</td>
                        <td>{percentage:.2f}%</td>
            """
            
            if similarity_matrix is not None:
                html_content += f"""
                        <td>{cohesion_scores[cluster_idx]:.4f}</td>
                """
                
            if 'stability_scores' in locals() and cluster_idx in stability_scores and stability_scores[cluster_idx]:
                stability = np.mean(stability_scores[cluster_idx])
                html_content += f"""
                        <td>{stability:.4f}</td>
                """
            
            html_content += """
                    </tr>
            """
        
        html_content += """
                </table>
                
                <h2>Conclusion</h2>
                <p>
                    This analysis highlights the best clusters identified by your algorithm, showcasing their 
                    distinctive features, internal cohesion, and stability. The visualizations demonstrate how 
                    well-separated these clusters are and which features contribute most to their formation.
                </p>
        """
        
        # Close tags
        html_content += """
            </div>
        </body>
        </html>
        """
        
        # Write the HTML file
        with open(f"{output_dir}/best_clusters_report.html", "w") as f:
            f.write(html_content)
        
        print(f"HTML report generated: {output_dir}/best_clusters_report.html")
    except Exception as e:
        print(f"Error generating HTML report: {e}")
    
    print(f"All best cluster visualizations saved to {output_dir}/")
    return True



In [ ]:
visualize_best_cluster_analysis(
    clusters=clusters, 
    densities=densities,
    features_df=features_df, 
    similarity_matrix=similarity,  # Pass the similarity matrix
    # labels=labels,  # Optional
    output_dir="best_cluster_analysis"  # Output directory
)